# 0. Setup

## python version

In [1]:
from platform import python_version
print(python_version())

3.7.9


## Display Size

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Librerie

In [3]:
# Scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mechanicalsoup as ms
import scrapy
import selenium
import lxml
import re
import requests
import pandas as pd
from pandas import compat
import numpy as np
import json
from requests import get
import pprint

# Matching
import Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Record Linkage
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonetic
#import html5lib

## Setup Selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 1. Scraping: `victoryroadvgc` 2022 Play! Pokémon Season Structure (Permitted, Restricted, Banned e Gigantamax pokemons)

Sulla pagina https://victoryroadvgc.com/2020-season/ è presente il regolamento completo della season 2022 di VGC.
In particolare estraggo:
   - Permitted Pokemon 
   - Restricted Pokemon (players may use two of the following Restricted Pokémon in their teams)
   - Banned Pokemon 
   - Gigantamax Pokemon
   
Nei primi 2 casi si farà riferimento al numero del **pokedex nazionale di ottava gen** -> il `#` nel dataset `pokedex` preso da `pokemon db` ma anche ai **pokedex regionali** -> quindi andranno poi integradi a dovere

## bs4 setup

In [5]:
victoryroadvgc_URL = 'https://victoryroadvgc.com/2020-season/'
victoryroadvgc_page = requests.get(victoryroadvgc_URL)

In [6]:
victoryroadvgc_soup = BeautifulSoup(victoryroadvgc_page.content, features='html.parser')

## Permitted e  Restricted

In [7]:
national_pokedex_rules = victoryroadvgc_soup.find_all("strong", string="National Pokédex:")

Permitted

In [8]:
national_pokedex_rules[0].parent.contents[1]

' #243–245, #252–260, #380–381, #480–482, #485, #488, #638–642, #645, #722–730, #785–788, #793–799 and #803–806'

Restricted

In [9]:
national_pokedex_rules[1].parent.contents[1]

' #150, #249-250, #382–384, #483–484, #487, #643–644, #646, #716–718, #789–792, #800, #888–890 and #898'

Definzione liste dei pokemon permessi nei vari pokedex

In [10]:
galar_permitted = list(range(1, 398))
isleofarmor_permitted = list(range(1, 211))
crowntundra_permitted = list(range(1, 210))
national_permitted = list(range(243, 246)) + list(range(252, 261)) + list(
    range(380, 382)) + list(range(480, 483)) + [485] + [488] + list(range(638, 643)) + [645] + list(
            range(722, 731)) + list(range(785, 789)) + list(range(793, 799)) + list(
                range(803, 807))

Definizione lista dei pokemon Restricted

In [11]:
national_restricted = [150] + list(range(249, 251)) + list(
    range(382, 385)) + list(range(483, 485)) + [487] + list(range(643, 645)) + [646] + list(
            range(716, 719)) + list(range(789, 793)) +[800] + list(
                range(888, 890)) + [898]

## Banned 

In [12]:
banned = ['Mew', 'Celebi', 'Jirachi', 'Victini', 'Keldeo (both forms)', 'Genesect (all forms)', 'Diancie', 'Volcanion', 'Magearna (both forms)', 'Marshadow', 'Zeraora', 'Meltan', 'Melmetal', 'Zarude (both forms)']

## Gigantamax Pokémon allowed 

In [13]:
gigantamax_allowed_soup = victoryroadvgc_soup.find("th", string = 'Gigantamax Pokémon allowed for Series 5, 7, 8, 9, 11 and 12').parent.parent.parent.find('tbody')

In [14]:
import re
gigantamax_allowed_list = re.split('(?=[A-Z])', gigantamax_allowed_soup.text)[1:-9]

In [15]:
gigantamax_allowed_list = gigantamax_allowed_list + ['Duraludon','Single Strike Style Urshifu','Rapid Strike Style Urshifu']

In [16]:
gigantamax_allowed_list

['Venusaur',
 'Charizard',
 'Blastoise',
 'Butterfree',
 'Pikachu',
 'Meowth',
 'Machamp',
 'Gengar',
 'Kingler',
 'Lapras',
 'Eevee',
 'Snorlax',
 'Garbodor',
 'Rillaboom',
 'Cinderace',
 'Inteleon',
 'Corviknight',
 'Orbeetle',
 'Drednaw',
 'Coalossal',
 'Flapple',
 'Appletun',
 'Sandaconda',
 'Toxtricity(both forms)',
 'Centiskorch',
 'Hatterene',
 'Grimmsnarl',
 'Alcremie(all forms)',
 'Copperajah',
 'Duraludon',
 'Single Strike Style Urshifu',
 'Rapid Strike Style Urshifu']

# 2. Integrazione

## G-Max Moves e Gigantamax Pokemon (integrazione su Pokemon e Moves)

In [17]:
path = 'C:/Users/giorg/OneDrive - Università degli Studi di Milano-Bicocca/Laurea Magistrale - Data Science/directory_progetti/Data-Management-Project//'

### Importazione e manipolazione

In [18]:
path = 'C:\\Users\\giorg\\OneDrive - Università degli Studi di Milano-Bicocca\\Laurea Magistrale - Data Science\\directory_progetti\\Data-Management-Project\\Dataset integrati\\'
pokemon_i_df = pd.read_csv(f'{path}pokemon_integrati.csv', index_col=0).drop(axis=1, labels='index')

In [19]:
pokemon_i_df.head()

,#,Name,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Variant,Varieties,sprite_url
0,1,Bulbasaur,1,Common,NaN,False,Grass,Poison,318,45,49,49,65,65,45,NaN,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,NaN,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,NaN,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,Mega Venusaur,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,4,Charmander,1,Common,NaN,False,Fire,NaN,309,39,52,43,60,50,65,NaN,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...


In [20]:
col = pokemon_i_df.pop("Variant")

In [21]:
pokemon_i_df.insert(2, "Variant", col)

In [22]:
pokemon_i_df.head()

,#,Name,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,NaN,1,Common,NaN,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,NaN,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,NaN,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Mega Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,4,Charmander,NaN,1,Common,NaN,False,Fire,NaN,309,39,52,43,60,50,65,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...


### Gigantamax: integrazione url sprite e integrazione al dataset principale pokemon

In [23]:
pokemon_gmax_df = pd.read_csv('./Scraping_datasets/bulbapedia/gigantamax_pkmn.csv', na_values=['NA', 'NaN'], index_col=0)

In [24]:
pokemon_gmax_df

,Pokémon,Type,Height,G-Max Move,G-Max Move Type
0,Charizard,Fire Flying,"91'10""+(28.0+ m)",G-Max Wildfire,Fire
1,Butterfree,Bug Flying,"55'09""+(17.0+ m)",G-Max Befuddle,Bug
2,Pikachu,Electric,"68'11""+(21.0+ m)",G-Max Volt Crash,Electric
3,Meowth,Normal,"108'03""+(33.0+ m)",G-Max Gold Rush,Normal
4,Machamp,Fighting,"82'00""+(25.0+ m)",G-Max Chi Strike,Fighting
5,Gengar,Ghost Poison,"65'07""+(20.0+ m)",G-Max Terror,Ghost
6,Kingler,Water,"62'04""+(19.0+ m)",G-Max Foam Burst,Water
7,Lapras,Water Ice,"78'09""+(24.0+ m)",G-Max Resonance,Ice
8,Eevee,Normal,"59'01""+(18.0+ m)",G-Max Cuddle,Normal
9,Snorlax,Normal,"114'10""+(35.0+ m)",G-Max Replenish,Normal


Correggo dei nomi

In [25]:
pokemon_gmax_df['Pokémon'].loc[pokemon_gmax_df['Pokémon'] == 'Toxtricity(Both forms)'] = 'Toxtricity'
pokemon_gmax_df['Pokémon'].loc[pokemon_gmax_df['Pokémon'] == 'Alcremie(Any form)'] = 'Alcremie'
pokemon_gmax_df['Pokémon'].loc[pokemon_gmax_df['Pokémon'] == 'Urshifu(Single Strike Style)'] = 'Urshifu (Single Strike)'
pokemon_gmax_df['Pokémon'].loc[pokemon_gmax_df['Pokémon'] == 'Urshifu(Rapid Strike Style)'] = 'Urshifu (Rapid Strike)'

In [26]:
moves_gmax_df = pd.read_csv('./Scraping_datasets/bulbapedia/gmax_moves.csv', na_values=['NA', 'NaN'], index_col=0)

In [27]:
pokemon_gmax_sprite_df = moves_gmax_df[['Gigantamax Pokémon', 'image_url']]

In [28]:
pokemon_gmax_df = pd.merge(pokemon_gmax_df,
                           pokemon_gmax_sprite_df,
                           how='inner',
                           left_on='Pokémon',
                           right_on='Gigantamax Pokémon')

In [29]:
pokemon_gmax_df.drop(axis=1, labels='Gigantamax Pokémon', inplace=True)

In [30]:
pokemon_gmax_df.insert(1, 'Variant', ['Gigantamax'] * len(pokemon_gmax_df))

In [31]:
pokemon_gmax_df[['Type1','Type2']] = pokemon_gmax_df['Type'].str.split(" ", 1, expand=True)
pokemon_gmax_df.drop(axis=1, labels='Type', inplace=True)

In [32]:
pokemon_gmax_df = pokemon_gmax_df.rename(columns={'Pokémon':'Name', 'image_url':'sprite_url'})
pokemon_gmax_df = pokemon_gmax_df[['Name', 'Variant', 'sprite_url', 'Type1', 'Type2']] 

In [33]:
pokemon_gmax_df.insert(1, 'Generation', [8] * len(pokemon_gmax_df))

Faccio un merge parziale con il dataset completo dei pokemon per ottenere il numero del pokedex nazionale (per Urshifu vado invece manualmente)

In [34]:
pokemon_gmax_df_Urshifu = pokemon_gmax_df[pokemon_gmax_df['Name'].str.contains(
    'Urshifu')]
pokemon_gmax_df_Urshifu['#'] = [892,892]  
pokemon_gmax_df_Urshifu

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Name,Generation,Variant,sprite_url,Type1,Type2,#
31,Urshifu (Single Strike),8,Gigantamax,https://archives.bulbagarden.net/media/upload/...,Fighting,Dark,892
32,Urshifu (Rapid Strike),8,Gigantamax,https://archives.bulbagarden.net/media/upload/...,Fighting,Water,892


In [35]:
pokemon_gmax_df = pd.merge(
    pokemon_gmax_df,
    pokemon_i_df[pokemon_i_df['Variant'].isnull()][['#', 'Name']],
    how='inner', on='Name')

Vado ad unire il dataset dei pokemon integrale e il dataset delle gigantamax

In [36]:
frame = [pokemon_i_df, pokemon_gmax_df, pokemon_gmax_df_Urshifu]
pokemon_i_df = pd.concat(frame)

In [37]:
pokemon_i_df = pokemon_i_df.convert_dtypes()
pokemon_i_df.sort_values(by = ['#'], inplace=True)

In [38]:
pokemon_i_df.reset_index(inplace=True)

In [39]:
pokemon_i_df.drop(axis=1, labels='index', inplace=True)

In [40]:
pokemon_i_df

,#,Name,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Mega Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur,Gigantamax,8,<NA>,<NA>,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,896,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1094,897,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1095,898,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1096,898,Calyrex,Shadow Rider,8,Legendary,<NA>,False,Psychic,Ghost,680,100,85,80,165,100,150,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


### G-max moves: integrazione gmax dell'API e gmax dello scraping (record linkage)

Dataset API

In [41]:
moves_gmax_API_df = pd.read_csv('./Dataset integrati/gmax_moves.csv').drop(axis=1, labels='Unnamed: 0')
moves_gmax_API_df.head()

,Name,Type,Power,Acc.,PP,Effect,Prob. (%),Cat.
0,G Max Befuddle,Bug,—,∞,5,"Butterfree-exclusive G-Max Move. Poisons, para...",100,—
1,G Max Cannonade,Water,—,∞,10,Blastoise-exclusive G-Max Move. Damages non-Wa...,—,—
2,G Max Centiferno,Fire,—,∞,5,Centiskorch-exclusive G-Max Move. Traps oppone...,100,—
3,G Max Chi Strike,Fighting,—,∞,5,Machamp-exclusive G-Max Move. Increases critic...,—,—
4,G Max Cuddle,Normal,—,∞,5,Eevee-exclusive G-Max Move. Infatuates opponents.,100,—


Dataset Scraping da bulbapedia

In [42]:
moves_gmax_df.head()

,Max Move,Gigantamax Pokémon,Type,Additional effect,image_url
0,G-Max Vine Lash,Venusaur,Grass,Inflicts damage for four turns on non-Grass-ty...,https://archives.bulbagarden.net/media/upload/...
1,G-Max Wildfire,Charizard,Fire,Inflicts damage for four turns on non-Fire-typ...,https://archives.bulbagarden.net/media/upload/...
2,G-Max Cannonade,Blastoise,Water,Inflicts damage for four turns on non-Water-ty...,https://archives.bulbagarden.net/media/upload/...
3,G-Max Befuddle,Butterfree,Bug,"Inflicts poison, paralysis, or sleep on all op...",https://archives.bulbagarden.net/media/upload/...
4,G-Max Volt Crash,Pikachu,Electric,Paralyzes all opponents,https://archives.bulbagarden.net/media/upload/...


#### Record Linkage

Setup

In [43]:
indexer = rl.Index()
indexer.full()

<Index>

In [44]:
candidates = indexer.index(moves_gmax_df, moves_gmax_API_df)

Comparing del nome

In [45]:
compare = rl.Compare()
compare.string('Max Move', 'Name', threshold=0.9, label='Name')
features = compare.compute(candidates, moves_gmax_df, moves_gmax_API_df)

Come atteso matchano tutte

In [46]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0      33
0.0    1056
dtype: int64

Estraggo i match potenziali

In [47]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

Mergio i due datasets

In [48]:
moves_gmax_i_df = pd.merge(potential_matches, moves_gmax_df, left_on="level_0", right_index=True)
moves_gmax_i_df = pd.merge(moves_gmax_i_df, moves_gmax_API_df.drop(axis=1, labels='Type'), left_on="level_1", right_index=True)

In [49]:
moves_gmax_i_df.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Name', 'Cat.', 'Effect'], inplace=True)

Sistemo nomi colonne e sostituisco i - con i Nan

In [50]:
moves_gmax_i_df = moves_gmax_i_df.replace(['—'],np.nan)

### Tabella ponte g-max moves/gigantamax pokemon

Estraggo le colonne del dataset dell'api utili come ponte

In [51]:
ponte_gmax_gigantamax = moves_gmax_i_df[['Max Move','Gigantamax Pokémon']]

In [52]:
ponte_gmax_gigantamax['Gigantamax Pokémon'] = ponte_gmax_gigantamax['Gigantamax Pokémon'] + ' Gigantamax'
ponte_gmax_gigantamax.to_csv('./Dataset integrati 2.0/ponte_moves_pokemon_GMAX_MOVE.csv', index=False)

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
ponte_gmax_gigantamax

,Max Move,Gigantamax Pokémon
0,G-Max Vine Lash,Venusaur Gigantamax
1,G-Max Wildfire,Charizard Gigantamax
2,G-Max Cannonade,Blastoise Gigantamax
3,G-Max Befuddle,Butterfree Gigantamax
4,G-Max Volt Crash,Pikachu Gigantamax
5,G-Max Gold Rush,Meowth Gigantamax
6,G-Max Chi Strike,Machamp Gigantamax
7,G-Max Terror,Gengar Gigantamax
8,G-Max Foam Burst,Kingler Gigantamax
9,G-Max Resonance,Lapras Gigantamax


### G-max moves: sistemazione ds integrato di moves integrazione con il dataset delle g-max ottenuto dallo scraping + api

Importazione dataset originale integrato, sistemazione delle colonne doppie 

In [54]:
moves_i_df = pd.read_csv('./Dataset integrati/moves_integrati.csv',
                         index_col=0).drop(axis=1, labels='index')
moves_i_df = moves_i_df.drop(axis=1,
                             labels=['Type_x', 'Power_x', 'Accuracy',
                                     'PP_x']).rename(columns={
                                         'Type_y': 'Type',
                                         'Power_y': 'Power',
                                         'PP_y': 'PP'
                                     })

Sostituzione dei dati vuoti da '-' a Nan

In [55]:
moves_i_df = moves_i_df.replace(['—'],np.nan)

In [56]:
moves_i_df.head(2)

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%),Learned_by
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,NaN,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,NaN,"[{'name': 'lycanroc-midday', 'url': 'https://p..."


Dataset ottenuto dall'integrazione di API e scraping

In [57]:
moves_gmax_i_df.head(2)

,Max Move,Gigantamax Pokémon,Type,Additional effect,image_url,Power,Acc.,PP,Prob. (%)
0,G-Max Vine Lash,Venusaur,Grass,Inflicts damage for four turns on non-Grass-ty...,https://archives.bulbagarden.net/media/upload/...,NaN,∞,10,NaN
1,G-Max Wildfire,Charizard,Fire,Inflicts damage for four turns on non-Fire-typ...,https://archives.bulbagarden.net/media/upload/...,NaN,∞,10,100


Vado a rinominare le colonne in modo da permettere la concatenazione e droppo quelle inutili

In [58]:
moves_gmax_i_df = moves_gmax_i_df.drop(axis = 1, labels=['image_url', 'Gigantamax Pokémon'])
moves_gmax_i_df.rename(columns={'Max Move':'Name','Additional effect':'Effect'}, inplace=True)
# Aggiungo una colonna relativa alla generazione di inserimento delle g-max
moves_gmax_i_df.insert(1, 'Introducted_in', [8] * len(moves_gmax_i_df))
frame = [moves_i_df, moves_gmax_i_df]
moves_i_df = pd.concat(frame).reset_index().drop(axis=1, labels='index')
# Converto float errati in interi
moves_i_df.replace(['—'],np.nan)
moves_i_df = moves_i_df.convert_dtypes()

In [59]:
moves_i_df.head(3)

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%),Learned_by
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>,"[{'name': 'lycanroc-midday', 'url': 'https://p..."
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."


### Dataset Ottenuti

In [60]:
pokemon_i_df # Pokemon Integrato
ponte_gmax_gigantamax # Tabella ponte
moves_i_df # Mosse integrato

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%),Learned_by
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>,"[{'name': 'lycanroc-midday', 'url': 'https://p..."
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."
3,Acid Armor,1,Poison,<NA>,<NA>,20,Status,Sharply raises user's Defense.,<NA>,"[{'name': 'tentacool', 'url': 'https://pokeapi..."
4,Acid Spray,5,Poison,40,100,20,Special,Sharply lowers opponent's Special Defense.,100,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."
...,...,...,...,...,...,...,...,...,...,...
816,G-Max Finale,8,Fairy,<NA>,∞,5,<NA>,Heals the user and its allies by 1/6 their max...,<NA>,<NA>
817,G-Max Steelsurge,8,Steel,<NA>,∞,5,<NA>,Scatters sharp spikes around the field Works l...,<NA>,<NA>
818,G-Max Depletion,8,Dragon,<NA>,∞,5,<NA>,Takes away 2 PP from the last move the target ...,<NA>,<NA>
819,G-Max One Blow,8,Dark,<NA>,∞,5,<NA>,Hits the target even if it is protected by a p...,<NA>,<NA>


## Pokemon: sistemazione ds integrato

### Sistemazione dei nomi dei pokemon

Creo l'identificativo 'Nome' che sarà la somma di Name e Variant e permetterà di identitficare unvicoamente il pokemon

In [61]:
pokemon_i_df.head()

,#,Name,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Mega Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur,Gigantamax,8,<NA>,<NA>,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...


Faccio dei controlli:
- Se non è una variante allora il suo nome coinciderà con il nome della specie
- Se Name è contenuto in variant allora il nome coinciderà con variant
- Altrimenti il nome è la somma di specie e variante

In [62]:
pokemon_complete_name = []
for index, row in pokemon_i_df.iterrows():
    if pd.isna(row['Variant']):
        pokemon_complete_name.append(row['Name'])
    elif row['Name'] in row['Variant']:
        pokemon_complete_name.append(row['Variant'])
    else:
        pokemon_complete_name.append(row['Name'] + ' ' + row['Variant'])

Converto il vecchio Name nel nuovo Specie

In [63]:
pokemon_i_df.rename(columns={'Name':'Species'}, inplace=True)

Inserisco la colonna relativa al nome completo

In [64]:
pokemon_i_df.insert(1, 'Name', pokemon_complete_name)

Rimuovo dalla variante il nome del pokemon

In [65]:
pokemon_variant_checked = []
for index, row in pokemon_i_df.iterrows():
    if pd.isna(row['Variant']):
        pokemon_variant_checked.append('Nan')
    elif row['Species'] in row['Variant']:
        pokemon_variant_checked.append(row['Variant'].replace(row['Species'], '').strip())
    else:
        pokemon_variant_checked.append(row['Variant'])

In [66]:
pokemon_i_df['Variant'] = pokemon_variant_checked
pokemon_i_df['Variant'] = pokemon_i_df['Variant'].replace(['Nan'],np.nan)
pokemon_i_df = pokemon_i_df.convert_dtypes()

### Sistemazione generazioni per alcune varianti

In [67]:
pokemon_i_df.loc[pokemon_i_df["Variant"].str.contains('Mega'), 'Generation'] = 6
pokemon_i_df.loc[pokemon_i_df["Variant"].str.contains('Alolan'), 'Generation'] = 7
pokemon_i_df.loc[pokemon_i_df["Variant"].str.contains('Galarian'), 'Generation'] = 8
pokemon_i_df.loc[pokemon_i_df["Variant"].str.contains('Primal'), 'Generation'] = 6

#pokemon_generation_checked = []
#for index, row in pokemon_i_df.iterrows():
#    if pd.isnull(row['Generation']):
#        pokemon_variant_checked.append('Nan')
#    elif pd.isnull(row['Variant']):
#        pokemon_variant_checked.append(row['Generation'])
#    elif row['Variant'] == 'Mega':
#        pokemon_generation_checked.append('6')
#    elif row['Variant'] == 'Alolan':
#        pokemon_generation_checked.append('7')
#    elif row['Variant'] == 'Galarian':
#        pokemon_generation_checked.append('8') 
#    elif row['Variant'] == 'Primal': #ArcheoGroudon e ArcheoKyogre
#        pokemon_generation_checked.append('6')
#    else:
#        pokemon_generation_checked.append(row['Generation'])

### Sistemazione evolved_from per la variante mega e gigantamax

Per le mega, gigantamax e primal evolves_from deve far rifermento alla specie (erroneamente nel dataset fa invece riferimento all'evoluzione precedente)

In [68]:
pokemon_i_df.loc[pokemon_i_df['Variant'].str.contains('Mega'), 'Evolves_from'] = pokemon_i_df['Species']
pokemon_i_df.loc[pokemon_i_df['Variant'].str.contains('Gigantamax'), 'Evolves_from'] = pokemon_i_df['Species']
pokemon_i_df.loc[pokemon_i_df['Variant'].str.contains('Primal'), 'Evolves_from'] = pokemon_i_df['Species']

### Eliminazione Varieties

In [184]:
pokemon_i_df.drop(axis=1, labels='Varieties', inplace=True)

### Cambio nome colonna sprites

In [186]:
pokemon_i_df.rename(columns={'sprite_url':'image_url'}, inplace=True)

### Dataset Ottenuto

In [69]:
pokemon_i_df.head(5)

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...


## Type, Ponte Type Chart (MOVE_EFFECTIVENESS_ON_POKEMON), Ponte Pokemon-Type (IS_OF_TYPE), Ponte Move-Type (MOVES_IS_TYPE)

### Type Entità

In [70]:
types_i_df = pd.read_csv("./API_datasets/types.csv")[['Name']]
types_i_df['Generation'] = [1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,2,6,2,3]
types_i_df

,Name,Generation
0,normal,1
1,fighting,1
2,flying,1
3,poison,1
4,ground,1
5,rock,1
6,bug,1
7,ghost,1
8,steel,2
9,fire,1


In [71]:
types_i_df.to_csv('./Dataset integrati 2.0/df_types.csv', index=False)

### Ponte type Chart

In [72]:
ponte_type_chart = pd.read_csv("./Scraping_datasets/pokemondb/typechart.csv",
                               index_col=0)
ponte_type_chart = ponte_type_chart.convert_dtypes()
ponte_type_chart

,Atk_Type/Pokemon_Type,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
0,Normal,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,½,0,<NA>,<NA>,½,<NA>
1,Fire,<NA>,½,½,<NA>,2,2,<NA>,<NA>,<NA>,<NA>,<NA>,2,½,<NA>,½,<NA>,2,<NA>
2,Water,<NA>,2,½,<NA>,½,<NA>,<NA>,<NA>,2,<NA>,<NA>,<NA>,2,<NA>,½,<NA>,<NA>,<NA>
3,Electric,<NA>,<NA>,2,½,½,<NA>,<NA>,<NA>,0,2,<NA>,<NA>,<NA>,<NA>,½,<NA>,<NA>,<NA>
4,Grass,<NA>,½,2,<NA>,½,<NA>,<NA>,½,2,½,<NA>,½,2,<NA>,½,<NA>,½,<NA>
5,Ice,<NA>,½,½,<NA>,2,½,<NA>,<NA>,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,½,<NA>
6,Fighting,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,½,<NA>,½,½,½,2,0,<NA>,2,2,½
7,Poison,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,½,½,<NA>,<NA>,<NA>,½,½,<NA>,<NA>,0,2
8,Ground,<NA>,2,<NA>,2,½,<NA>,<NA>,2,<NA>,0,<NA>,½,2,<NA>,<NA>,<NA>,2,<NA>
9,Flying,<NA>,<NA>,<NA>,½,2,<NA>,2,<NA>,<NA>,<NA>,<NA>,2,½,<NA>,<NA>,<NA>,½,<NA>


Vado a normalizzare il dataset trasformandolo in due colonne

In [73]:
ponte_type_chart = pd.melt(
    ponte_type_chart,
    id_vars=['Atk_Type/Pokemon_Type'],
    value_vars=ponte_type_chart.columns.values.tolist()[1:],
    var_name='Def. Pokemon Type',
    value_name='Damage Multiplier').rename(
        columns={'Atk_Type/Pokemon_Type': 'Atk. Move Type'})

In [74]:
ponte_type_chart

,Atk. Move Type,Def. Pokemon Type,Damage Multiplier
0,Normal,Normal,<NA>
1,Fire,Normal,<NA>
2,Water,Normal,<NA>
3,Electric,Normal,<NA>
4,Grass,Normal,<NA>
...,...,...,...
319,Ghost,Fairy,<NA>
320,Dragon,Fairy,0
321,Dark,Fairy,½
322,Steel,Fairy,2


Aggiungo il parametro `effectiveness`

In [75]:
conditions = [
    ponte_type_chart['Damage Multiplier'].isnull(),
    ponte_type_chart['Damage Multiplier'] == '0',
    ponte_type_chart['Damage Multiplier'] == 0,
    ponte_type_chart['Damage Multiplier'] == '½',
    ponte_type_chart['Damage Multiplier'] == '2',
    ponte_type_chart['Damage Multiplier'] == 2
]

outputs = [
    'Normal (100%)', 'Normal (100%)', 'No effect (0%)',
    'Not very effective (50%)', 'Super-effective (200%)',
    'Super-effective (200%)'
]

ponte_type_chart['Effectiveness'] = np.select(conditions, outputs)
ponte_type_chart

,Atk. Move Type,Def. Pokemon Type,Damage Multiplier,Effectiveness
0,Normal,Normal,<NA>,Normal (100%)
1,Fire,Normal,<NA>,Normal (100%)
2,Water,Normal,<NA>,Normal (100%)
3,Electric,Normal,<NA>,Normal (100%)
4,Grass,Normal,<NA>,Normal (100%)
...,...,...,...,...
319,Ghost,Fairy,<NA>,Normal (100%)
320,Dragon,Fairy,0,Normal (100%)
321,Dark,Fairy,½,Not very effective (50%)
322,Steel,Fairy,2,Super-effective (200%)


In [76]:
ponte_type_chart.to_csv(
    './Dataset integrati 2.0/ponte_type_type_MOVE_EFFECTIVENESS_ON_POKEMON.csv',
    index=False)

### Ponte Pokemon-Type

Estraggo dal dataset dei pokemon i soli nomi e tipi

In [77]:
import warnings
warnings.filterwarnings(action='once')
ponte_pokemon_type = pokemon_i_df[['Name', 'Type1', 'Type2']]
ponte_pokemon_type['Type'] = ponte_pokemon_type[['Type1','Type2']].values.tolist()
ponte_pokemon_type.drop(axis=1, labels=['Type1', 'Type2'], inplace=True)
ponte_pokemon_type = ponte_pokemon_type.explode('Type')
ponte_pokemon_type = ponte_pokemon_type[~ponte_pokemon_type['Type'].isnull()]
ponte_pokemon_type

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Name,Type
0,Bulbasaur,Grass
0,Bulbasaur,Poison
1,Ivysaur,Grass
1,Ivysaur,Poison
2,Venusaur,Grass
...,...,...
1095,Calyrex,Grass
1096,Calyrex Shadow Rider,Psychic
1096,Calyrex Shadow Rider,Ghost
1097,Calyrex Ice Rider,Psychic


In [78]:
ponte_pokemon_type.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_type_IS_OF_TYPE.csv',
    index=False)

### Ponte MoveType-Type (MOVES_IS_TYPE)

In [79]:
ponte_move_type_MOVES_IS_TYPE = moves_i_df[['Name', 'Type']]
ponte_move_type_MOVES_IS_TYPE = ponte_move_type_MOVES_IS_TYPE[
    ~ponte_move_type_MOVES_IS_TYPE['Type'].isnull()]
ponte_move_type_MOVES_IS_TYPE

,Name,Type
0,Absorb,Grass
1,Accelerock,Rock
2,Acid,Poison
3,Acid Armor,Poison
4,Acid Spray,Poison
...,...,...
816,G-Max Finale,Fairy
817,G-Max Steelsurge,Steel
818,G-Max Depletion,Dragon
819,G-Max One Blow,Dark


In [80]:
ponte_move_type_MOVES_IS_TYPE.to_csv(
    './Dataset integrati 2.0/ponte_move_type_MOVES_IS_TYPE.csv',
    index=False)

## Pokemon: Sistemazione ABILITIES e creazione tabelle ponte: EVOLVES_FROM (pokemon-Pokemon), MAY_HAS (Pokemon-Ability), MAY_LEARN (Pokemon-Move), HAS_VARIANT (Pokemon-Pokemon)

### EVOLVES_FROM

In [81]:
ponte_pokemon_pokemon_EVOLVES_FROM = pokemon_i_df[
    ~pd.isnull(pokemon_i_df['Evolves_from'])][['Name', 'Evolves_from']]
ponte_pokemon_pokemon_EVOLVES_FROM.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_pokemon_EVOLVES_FROM.csv',
    index=False)

### HAS_VARIANT

In [187]:
pokemon_i_df.head(2)

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,image_url,VGC2022_rules
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/sword-shield...,Permitted
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/sword-shield...,Permitted


In [83]:
ponte_pokemon_pokemon_HAS_VARIANT = pd.merge(
    pokemon_i_df[pokemon_i_df['Variant'].isnull()][['Name', 'Species']],
    pokemon_i_df[~pokemon_i_df['Variant'].isnull()][['Name', 'Species']],
    on='Species',
    how='inner').rename(columns={
        'Name_x': 'Non-Variant Pkm Name',
        'Name_y': 'Variant Pkm Name'
    }).drop(axis=1, labels='Species')
ponte_pokemon_pokemon_HAS_VARIANT

,Non-Variant Pkm Name,Variant Pkm Name
0,Venusaur,Mega Venusaur
1,Venusaur,Venusaur Gigantamax
2,Charizard,Mega Charizard Y
3,Charizard,Charizard Gigantamax
4,Charizard,Mega Charizard X
...,...,...
148,Copperajah,Copperajah Gigantamax
149,Duraludon,Duraludon Gigantamax
150,Eternatus,Eternatus Eternamax
151,Calyrex,Calyrex Shadow Rider


In [84]:
ponte_pokemon_pokemon_HAS_VARIANT.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_pokemon_HAS_VARIANT.csv',
    index=False)

### MAY_LEARN

#### Importazione e pulizia del dataset ponte MAY_LEARN

In [85]:
ponte_pokemon_moves_MAY_LEARN = pd.read_csv('./Tabelle ponte/ponte_move_poke.csv', index_col=0)
#ponte_pokemon_moves_MAY_LEARN

Pulizia stringhe

In [86]:
# Elimino trattini tra i nomi
ponte_pokemon_moves_MAY_LEARN['Pokemon'] = rl.preprocessing.clean(
    ponte_pokemon_moves_MAY_LEARN['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
ponte_pokemon_moves_MAY_LEARN['Pokemon'] = ponte_pokemon_moves_MAY_LEARN['Pokemon'].str.title()

In [87]:
ponte_pokemon_moves_MAY_LEARN.head(5)

,Move,Pokemon
0,Absorb,Zubat
1,Absorb,Golbat
2,Absorb,Oddish
3,Absorb,Gloom
4,Absorb,Vileplume


Faccio una serie di check:
- A quanto pare il matching serve farlo solo sui nomi dei pokemon (tutte e 788 le mosse della tabella ponte si trovano identicamente del dataset delle mosse)
- Mancano delle mosse (le G-Max) che però hanno la loro relazionbe specifica

In [88]:
moves_i_df.shape

(821, 10)

In [89]:
sum(moves_i_df['Name'].isin(ponte_pokemon_moves_MAY_LEARN['Move']))

788

In [90]:
len(ponte_pokemon_moves_MAY_LEARN['Move'].unique())

788

In [91]:
#moves_i_df[~moves_i_df['Name'].isin(ponte_pokemon_moves_MAY_LEARN['Move'])]['Name']

Procedo quindi con il matching tra i nomi dei pokemon

In [92]:
len(ponte_pokemon_moves_MAY_LEARN['Pokemon'].unique())

1093

Ci sono 1093 pokemon, 2 in meno rispetto a quello dei pokemon integrato

#### Record Linkage

In [93]:
pokemon_i_df

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,896,Glastrier,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1094,897,Spectrier,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1095,898,Calyrex,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1096,898,Calyrex Shadow Rider,Calyrex,Shadow Rider,8,Legendary,<NA>,False,Psychic,Ghost,680,100,85,80,165,100,150,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


In [94]:
ponte_pokemon_moves_MAY_LEARN

,Move,Pokemon
0,Absorb,Zubat
1,Absorb,Golbat
2,Absorb,Oddish
3,Absorb,Gloom
4,Absorb,Vileplume
...,...,...
78177,Zen Headbutt,Calyrex Shadow
78178,Zing Zap,Togedemaru
78179,Zing Zap,Pincurchin
78180,Zing Zap,Togedemaru Totem


Essendo il dataset troppo grande procedo elaborando i soli valori unici, per poi generare una tabella ponte e ricongiungermi ai 781982 record iniziali

In [95]:
ponte_pokemon_moves_MAY_LEARN_unique = pd.DataFrame(ponte_pokemon_moves_MAY_LEARN['Pokemon'].unique(), columns=['Pokemon'])

In [96]:
ponte_pokemon_moves_MAY_LEARN_unique

,Pokemon
0,Zubat
1,Golbat
2,Oddish
3,Gloom
4,Vileplume
...,...
1088,Unown
1089,Blipbug
1090,Smeargle
1091,Cosmog


Setup

In [97]:
indexer = rl.Index()
indexer.full()

<Index>

In [98]:
candidates = indexer.index(ponte_pokemon_moves_MAY_LEARN_unique, pokemon_i_df)
len(candidates)

1200114

Comparing del nome

In [99]:
compare = rl.Compare()
compare.string('Pokemon', 'Name', threshold=0.85, label='Name', method='qgram')
features = compare.compute(candidates, ponte_pokemon_moves_MAY_LEARN_unique,
                           pokemon_i_df)

Come atteso matchano tutte

In [100]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0        940
0.0    1199174
dtype: int64

Estraggo i match potenziali

In [101]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

In [102]:
potential_matches

,level_0,level_1,Score
0,0,59,1.0
1,1,60,1.0
2,2,61,1.0
3,3,62,1.0
4,4,63,1.0
...,...,...,...
935,1088,265,1.0
936,1089,999,1.0
937,1090,306,1.0
938,1091,956,1.0


Mergio i due datasets

In [103]:
ponte_pokemon_moves_MAY_LEARN_unique_ = pd.merge(potential_matches, ponte_pokemon_moves_MAY_LEARN_unique, left_on="level_0", right_index=True)
ponte_pokemon_moves_MAY_LEARN_unique_ = pd.merge(ponte_pokemon_moves_MAY_LEARN_unique_, pokemon_i_df[['Name', 'Variant']], left_on="level_1", right_index=True)

In [104]:
pd.set_option('display.max_rows', 30)
ponte_pokemon_moves_MAY_LEARN_unique_

,level_0,level_1,Score,Pokemon,Name,Variant
0,0,59,1.0,Zubat,Zubat,<NA>
1,1,60,1.0,Golbat,Golbat,<NA>
2,2,61,1.0,Oddish,Oddish,<NA>
3,3,62,1.0,Gloom,Gloom,<NA>
4,4,63,1.0,Vileplume,Vileplume,<NA>
...,...,...,...,...,...,...
935,1088,265,1.0,Unown,Unown,<NA>
936,1089,999,1.0,Blipbug,Blipbug,<NA>
937,1090,306,1.0,Smeargle,Smeargle,<NA>
938,1091,956,1.0,Cosmog,Cosmog,<NA>


In [105]:
ponte_pokemon_moves_MAY_LEARN_unique_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Variant'], inplace=True)
ponte_pokemon_moves_MAY_LEARN_unique_

,Pokemon,Name
0,Zubat,Zubat
1,Golbat,Golbat
2,Oddish,Oddish
3,Gloom,Gloom
4,Vileplume,Vileplume
...,...,...
935,Unown,Unown
936,Blipbug,Blipbug
937,Smeargle,Smeargle
938,Cosmog,Cosmog


Vado ora a joinare il datataset ponte con questa tabella ponte, rigenerando la tabella ponte originale con circa 78000 record

In [106]:
ponte_pokemon_moves_MAY_LEARN = pd.merge(ponte_pokemon_moves_MAY_LEARN, ponte_pokemon_moves_MAY_LEARN_unique_, on = 'Pokemon', how = 'inner')

Elimino la colonna con il nome non matchato (quella originale dell'api)

In [107]:
ponte_pokemon_moves_MAY_LEARN.drop(axis=1, labels='Pokemon', inplace=True)
ponte_pokemon_moves_MAY_LEARN.rename(columns={'Name':'Pokemon'}, inplace=True)

Salvataggio

In [108]:
ponte_pokemon_moves_MAY_LEARN.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_moves_MAY_LEARN.csv',
    index=False)

### ABILITIES (salvataggio)

In [109]:
abilities_i_df = pd.read_csv('./Dataset integrati/Abilities_integrati.csv', index_col=0).drop(axis = 1, labels = ['index', 'Pokemon'])

In [110]:
abilities_i_df

,Name,Generation,Description
0,Adaptability,4,Powers up moves of the same type.
1,Aerilate,6,Turns Normal-type moves into Flying-type moves.
2,Aftermath,4,Damages the attacker landing the finishing hit.
3,Air Lock,3,Eliminates the effects of weather.
4,Analytic,5,Boosts move power when the Pokémon moves last.
...,...,...,...
262,White Smoke,3,Prevents other Pokémon from lowering its stats.
263,Wimp Out,7,Switches out when HP drops below half.
264,Wonder Guard,3,Only supereffective moves will hit.
265,Wonder Skin,5,Makes status-changing moves more likely to miss.


In [111]:
abilities_i_df.to_csv(
    './Dataset integrati 2.0/df_abilities.csv',
    index=False)

### MAY_HAS

#### Importazione e pulizia del dataset ponte MAY_LEARN e Abilities

In [112]:
ponte_pokemon_ability_MAY_HAS = pd.read_csv('./Tabelle ponte/ponte_abil_poke.csv', index_col=0)
ponte_pokemon_ability_MAY_HAS

,Ability,Pokemon,Hidden
0,Adaptability,eevee,False
1,Adaptability,corphish,True
2,Adaptability,crawdaunt,True
3,Adaptability,feebas,True
4,Adaptability,porygon-z,False
...,...,...,...
2528,Wonder Skin,bruxish,True
2529,Zen Mode,darmanitan-standard,True
2530,Zen Mode,darmanitan-zen,True
2531,Zen Mode,darmanitan-galar-standard,True


Pulizia stringhe

In [113]:
# Elimino trattini tra i nomi
ponte_pokemon_ability_MAY_HAS['Pokemon'] = rl.preprocessing.clean(
    ponte_pokemon_ability_MAY_HAS['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
ponte_pokemon_ability_MAY_HAS['Pokemon'] = ponte_pokemon_ability_MAY_HAS['Pokemon'].str.title()

In [114]:
ponte_pokemon_ability_MAY_HAS.head(5)

,Ability,Pokemon,Hidden
0,Adaptability,Eevee,False
1,Adaptability,Corphish,True
2,Adaptability,Crawdaunt,True
3,Adaptability,Feebas,True
4,Adaptability,Porygon Z,False


Faccio una serie di check:
- A quanto pare il matching serve farlo solo sui nomi dei pokemon (tutte e 788 le mosse della tabella ponte si trovano identicamente del dataset delle mosse)
- Mancano delle mosse (le G-Max) che però hanno la loro relazionbe specifica

In [115]:
abilities_i_df.shape

(267, 3)

In [116]:
sum(abilities_i_df['Name'].isin(ponte_pokemon_ability_MAY_HAS['Ability']))

267

In [117]:
len(ponte_pokemon_ability_MAY_HAS['Ability'].unique())

267

Ci sono 1093 pokemon, 2 in meno rispetto a quello dei pokemon integrato

#### Record Linkage

In [118]:
pokemon_i_df

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,896,Glastrier,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1094,897,Spectrier,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1095,898,Calyrex,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1096,898,Calyrex Shadow Rider,Calyrex,Shadow Rider,8,Legendary,<NA>,False,Psychic,Ghost,680,100,85,80,165,100,150,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


In [119]:
ponte_pokemon_ability_MAY_HAS

,Ability,Pokemon,Hidden
0,Adaptability,Eevee,False
1,Adaptability,Corphish,True
2,Adaptability,Crawdaunt,True
3,Adaptability,Feebas,True
4,Adaptability,Porygon Z,False
...,...,...,...
2528,Wonder Skin,Bruxish,True
2529,Zen Mode,Darmanitan Standard,True
2530,Zen Mode,Darmanitan Zen,True
2531,Zen Mode,Darmanitan Galar Standard,True


Essendo il dataset troppo grande procedo elaborando i soli valori unici, per poi generare una tabella ponte e ricongiungermi ai 781982 record iniziali

In [120]:
ponte_pokemon_ability_MAY_HAS_unique = pd.DataFrame(ponte_pokemon_ability_MAY_HAS['Pokemon'].unique(), columns=['Pokemon'])

In [121]:
ponte_pokemon_ability_MAY_HAS_unique

,Pokemon
0,Eevee
1,Corphish
2,Crawdaunt
3,Feebas
4,Porygon Z
...,...
1121,Dewpider
1122,Araquanid
1123,Araquanid Totem
1124,Wimpod


Setup

In [122]:
indexer = rl.Index()
indexer.full()

<Index>

In [123]:
candidates = indexer.index(ponte_pokemon_ability_MAY_HAS_unique, pokemon_i_df)
len(candidates)

1236348

Comparing del nome

In [124]:
compare = rl.Compare()
compare.string('Pokemon', 'Name', threshold=0.85, label='Name', method='qgram')
features = compare.compute(candidates, ponte_pokemon_ability_MAY_HAS_unique,
                           pokemon_i_df)

Come atteso matchano tutte

In [125]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0        940
0.0    1235408
dtype: int64

Estraggo i match potenziali

In [126]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

In [127]:
potential_matches

,level_0,level_1,Score
0,0,187,1.0
1,1,427,1.0
2,2,428,1.0
3,3,435,1.0
4,6,835,1.0
...,...,...,...
935,1120,879,1.0
936,1121,917,1.0
937,1122,918,1.0
938,1124,933,1.0


Mergio i due datasets

In [128]:
ponte_pokemon_ability_MAY_HAS_unique_ = pd.merge(potential_matches, ponte_pokemon_ability_MAY_HAS_unique, left_on="level_0", right_index=True)
ponte_pokemon_ability_MAY_HAS_unique_ = pd.merge(ponte_pokemon_ability_MAY_HAS_unique_, pokemon_i_df[['Name', 'Variant']], left_on="level_1", right_index=True)

In [129]:
pd.set_option('display.max_rows', 30)
ponte_pokemon_ability_MAY_HAS_unique_

,level_0,level_1,Score,Pokemon,Name,Variant
0,0,187,1.0,Eevee,Eevee,<NA>
1,1,427,1.0,Corphish,Corphish,<NA>
2,2,428,1.0,Crawdaunt,Crawdaunt,<NA>
3,3,435,1.0,Feebas,Feebas,<NA>
4,6,835,1.0,Skrelp,Skrelp,<NA>
...,...,...,...,...,...,...
935,1120,879,1.0,Volcanion,Volcanion,<NA>
936,1121,917,1.0,Dewpider,Dewpider,<NA>
937,1122,918,1.0,Araquanid,Araquanid,<NA>
938,1124,933,1.0,Wimpod,Wimpod,<NA>


In [130]:
ponte_pokemon_ability_MAY_HAS_unique_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Variant'], inplace=True)
ponte_pokemon_ability_MAY_HAS_unique_

,Pokemon,Name
0,Eevee,Eevee
1,Corphish,Corphish
2,Crawdaunt,Crawdaunt
3,Feebas,Feebas
4,Skrelp,Skrelp
...,...,...
935,Volcanion,Volcanion
936,Dewpider,Dewpider
937,Araquanid,Araquanid
938,Wimpod,Wimpod


Vado ora a joinare il datataset ponte con questa tabella ponte, rigenerando la tabella ponte originale con circa 78000 record

In [131]:
ponte_pokemon_ability_MAY_HAS = pd.merge(ponte_pokemon_ability_MAY_HAS, ponte_pokemon_ability_MAY_HAS_unique_, on = 'Pokemon', how = 'inner')
ponte_pokemon_ability_MAY_HAS

,Ability,Pokemon,Hidden,Name
0,Adaptability,Eevee,False,Eevee
1,Anticipation,Eevee,True,Eevee
2,Run Away,Eevee,False,Eevee
3,Adaptability,Corphish,True,Corphish
4,Hyper Cutter,Corphish,False,Corphish
...,...,...,...,...
2170,Water Bubble,Dewpider,False,Dewpider
2171,Water Absorb,Araquanid,True,Araquanid
2172,Water Bubble,Araquanid,False,Araquanid
2173,Wimp Out,Wimpod,False,Wimpod


Elimino la colonna con il nome non matchato (quella originale dell'api)

In [132]:
ponte_pokemon_ability_MAY_HAS.drop(axis=1, labels='Pokemon', inplace=True)
ponte_pokemon_ability_MAY_HAS.rename(columns={'Name':'Pokemon'}, inplace=True)

In [133]:
ponte_pokemon_ability_MAY_HAS

,Ability,Hidden,Pokemon
0,Adaptability,False,Eevee
1,Anticipation,True,Eevee
2,Run Away,False,Eevee
3,Adaptability,True,Corphish
4,Hyper Cutter,False,Corphish
...,...,...,...
2170,Water Bubble,False,Dewpider
2171,Water Absorb,True,Araquanid
2172,Water Bubble,False,Araquanid
2173,Wimp Out,False,Wimpod


Salvataggio

In [134]:
ponte_pokemon_ability_MAY_HAS.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_ability_MAY_HAS.csv',
    index=False)

## Matching e integrazione pokemon, nuova colonna `VGC2022_rules`: Permitted, Restricted, Banned, Allowed (Gigantamax)

### Importazione pokedex regionali

In [135]:
# Funzione per eliminare i carattero giapponesi dalle stringhe
import string
ascii = set(string.printable)   

def remove_non_ascii(s):
    return ''.join(list(filter(lambda x: x in ascii, s)))

In [136]:
isleofarmor_pokedex = pd.read_csv('./Scraping_datasets/serebii/isleofarmor_pokedex.csv', index_col=0)
# Sistemazione numero del pokedex
isleofarmor_pokedex['No.'] = isleofarmor_pokedex['No.'].str.lstrip('#').astype(int)
# Sistemazione nomi
isleofarmor_pokedex['Name'] = isleofarmor_pokedex['Name'].apply(remove_non_ascii)
isleofarmor_pokedex['Name'].loc[isleofarmor_pokedex['Name'] == 'Slowbro'] = 'Galarian Slowbro'
isleofarmor_pokedex['Name'].loc[isleofarmor_pokedex['Name'] == 'Slowpoke'] = 'Galarian Slowpoke'
isleofarmor_pokedex

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd,sprite_url
0,1,Galarian Slowpoke,Gluttony Own Tempo Regenerator,90,65,65,40,40,15,https://www.serebii.net//swordshield/pokemon/s...
1,2,Galarian Slowbro,Quick Draw Own Tempo Regenerator,95,100,95,100,70,30,https://www.serebii.net//swordshield/pokemon/s...
2,3,Slowking,Curious Medicine Own Tempo Regenerator,95,65,80,110,110,30,https://www.serebii.net//swordshield/pokemon/s...
3,4,Buneary,Run Away Klutz Limber,55,66,44,44,56,85,https://www.serebii.net//swordshield/pokemon/s...
4,5,Lopunny,Cute Charm Klutz Limber,65,76,84,54,96,105,https://www.serebii.net//swordshield/pokemon/s...
...,...,...,...,...,...,...,...,...,...,...
206,207,Ditto,Limber Imposter,48,48,48,48,48,48,https://www.serebii.net//swordshield/pokemon/s...
207,208,Porygon,Trace Download Analytic,65,60,70,85,75,40,https://www.serebii.net//swordshield/pokemon/s...
208,209,Porygon2,Trace Download Analytic,85,80,90,105,95,60,https://www.serebii.net//swordshield/pokemon/s...
209,210,Porygon-Z,Adaptability Download Analytic,85,80,70,135,75,90,https://www.serebii.net//swordshield/pokemon/s...


In [137]:
swordandshield_pokedex = pd.read_csv('./Scraping_datasets/serebii/swordandshield_pokedex.csv', index_col=0)
# Sistemazione numero del pokedex
swordandshield_pokedex['No.'] = swordandshield_pokedex['No.'].str.lstrip('#').astype(int)
# Sistemazione nomi
swordandshield_pokedex['Name'] = swordandshield_pokedex['Name'].apply(remove_non_ascii)
# Correggo nomi galarian
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Zigzagoon'] = 'Galarian Zigzagoon'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Linoone'] = 'Galarian Linoone'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Meowth'] = 'Galarian Meowth'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Farfetch’d'] = 'Galarian Farfetch’d'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Stunfisk'] = 'Galarian Stunfisk'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Corsola'] = 'Galarian Corsola'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Weezing'] = 'Galarian Weezing'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Yamask'] = 'Galarian Yamask'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Ponyta'] = 'Galarian Ponyta'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Rapidash'] = 'Galarian Rapidash'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Mr. Mime'] = 'Galarian Mr. Mime'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Darumaka'] = 'Galarian Darumaka'
swordandshield_pokedex['Name'].loc[swordandshield_pokedex['Name'] == 'Darmanitan'] = 'Galarian Darmanitan'
swordandshield_pokedex.head(5)

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd,sprite_url
0,1,Grookey,Overgrow Grassy Surge,50,65,50,40,40,65,https://www.serebii.net//swordshield/pokemon/s...
1,2,Thwackey,Overgrow Grassy Surge,70,85,70,55,60,80,https://www.serebii.net//swordshield/pokemon/s...
2,3,Rillaboom,Overgrow Grassy Surge,100,125,90,60,70,85,https://www.serebii.net//swordshield/pokemon/s...
3,4,Scorbunny,Blaze Libero,50,71,40,40,40,69,https://www.serebii.net//swordshield/pokemon/s...
4,5,Raboot,Blaze Libero,65,86,60,55,60,94,https://www.serebii.net//swordshield/pokemon/s...


In [138]:
thecrowntundra_pokedex = pd.read_csv('./Scraping_datasets/serebii/thecrowntundra_pokedex.csv', index_col=0)

# Sistemazione numero del pokedex
thecrowntundra_pokedex['No.'] = thecrowntundra_pokedex['No.'].str.lstrip('#').astype(int)
# Sistemazione nomi
thecrowntundra_pokedex['Name'] = thecrowntundra_pokedex['Name'].apply(remove_non_ascii)
# Correggo nomi galarian
thecrowntundra_pokedex['Name'].loc[thecrowntundra_pokedex['Name'] == 'Mr. Mime'] = 'Galarian Mr. Mime'
thecrowntundra_pokedex['Name'].loc[thecrowntundra_pokedex['Name'] == 'Zigzagoon'] = 'Galarian Zigzagoon'
thecrowntundra_pokedex['Name'].loc[thecrowntundra_pokedex['Name'] == 'Darumaka'] = 'Galarian Darumaka'
thecrowntundra_pokedex['Name'].loc[thecrowntundra_pokedex['Name'] == 'Farfetch’d'] = 'Galarian Darmanitan'
thecrowntundra_pokedex['Name'].loc[thecrowntundra_pokedex['Name'] == 'Stunfisk'] = 'Galarian Ponyta'
thecrowntundra_pokedex['Name'].loc[thecrowntundra_pokedex['Name'] == 'Corsola'] = 'Galarian Rapidash'

thecrowntundra_pokedex.head(5)

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd,sprite_url
0,1,Snom,Shield Dust Ice Scales,30,25,35,45,30,20,https://www.serebii.net//swordshield/pokemon/s...
1,2,Frosmoth,Shield Dust Ice Scales,70,65,60,125,90,65,https://www.serebii.net//swordshield/pokemon/s...
2,3,Wooloo,Fluffy Run Away Bulletproof,42,40,55,40,45,48,https://www.serebii.net//swordshield/pokemon/s...
3,4,Dubwool,Fluffy Steadfast Bulletproof,72,80,100,60,90,88,https://www.serebii.net//swordshield/pokemon/s...
4,5,Skwovet,Cheek Pouch Gluttony,70,55,55,35,35,25,https://www.serebii.net//swordshield/pokemon/s...


### Permitted, Restricted, Banned e Allowed Lists

Permitted (#)

In [139]:
galar_permitted
isleofarmor_permitted
crowntundra_permitted
national_permitted

[243,
 244,
 245,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 380,
 381,
 480,
 481,
 482,
 485,
 488,
 638,
 639,
 640,
 641,
 642,
 645,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 730,
 785,
 786,
 787,
 788,
 793,
 794,
 795,
 796,
 797,
 798,
 803,
 804,
 805,
 806]

Restricted (#)

In [140]:
national_restricted

[150,
 249,
 250,
 382,
 383,
 384,
 483,
 484,
 487,
 643,
 644,
 646,
 716,
 717,
 718,
 789,
 790,
 791,
 792,
 800,
 888,
 889,
 898]

Banned (Name)

In [141]:
banned

['Mew',
 'Celebi',
 'Jirachi',
 'Victini',
 'Keldeo (both forms)',
 'Genesect (all forms)',
 'Diancie',
 'Volcanion',
 'Magearna (both forms)',
 'Marshadow',
 'Zeraora',
 'Meltan',
 'Melmetal',
 'Zarude (both forms)']

Gigantamax Pokémon allowed (Names)

In [142]:
len(gigantamax_allowed_list)

32

### Conversione di tutte le liste in Nomi dei pokemon

#### Permitted

##### National

Estraggo i nomi dei soli pokemon contenuti nella lista dei permitted

In [143]:
national_permitted_ = list(pokemon_i_df[pokemon_i_df['#'].isin(national_permitted) & pd.isnull(pokemon_i_df['Variant'])]['Name'])
len(national_permitted_)

41

##### Isle of armor

In [144]:
isleofarmor_permitted_ = list(isleofarmor_pokedex[isleofarmor_pokedex['No.'].isin(isleofarmor_permitted)]['Name'])
len(isleofarmor_permitted_)

210

##### swordandshield (galar)

In [145]:
galar_permitted_ = list(swordandshield_pokedex[swordandshield_pokedex['No.'].isin(galar_permitted)]['Name'])
len(galar_permitted_)

397

##### crowntundra

In [146]:
crowntundra_permitted_ = list(thecrowntundra_pokedex[thecrowntundra_pokedex['No.'].isin(crowntundra_permitted)]['Name'])
len(crowntundra_permitted_)

209

##### Lista finale

In [147]:
permitted_VGC2022 = national_permitted_ + isleofarmor_permitted_ + galar_permitted_ + crowntundra_permitted_

Elimino duplicati

In [148]:
permitted_VGC2022 = list(dict.fromkeys(permitted_VGC2022))

In [149]:
len(permitted_VGC2022)

622

In [162]:
permitted_VGC2022

['Raikou',
 'Entei',
 'Suicune',
 'Treecko',
 'Grovyle',
 'Sceptile',
 'Torchic',
 'Combusken',
 'Blaziken',
 'Mudkip',
 'Marshtomp',
 'Swampert',
 'Latias',
 'Latios',
 'Uxie',
 'Mesprit',
 'Azelf',
 'Heatran',
 'Cresselia',
 'Cobalion',
 'Terrakion',
 'Virizion',
 'Rowlet',
 'Dartrix',
 'Decidueye',
 'Litten',
 'Torracat',
 'Incineroar',
 'Popplio',
 'Brionne',
 'Primarina',
 'Nihilego',
 'Buzzwole',
 'Pheromosa',
 'Xurkitree',
 'Celesteela',
 'Kartana',
 'Poipole',
 'Naganadel',
 'Stakataka',
 'Blacephalon',
 'Galarian Slowpoke',
 'Galarian Slowbro',
 'Slowking',
 'Buneary',
 'Lopunny',
 'Happiny',
 'Chansey',
 'Blissey',
 'Skwovet',
 'Greedent',
 'Igglybuff',
 'Jigglypuff',
 'Wigglytuff',
 'Blipbug',
 'Dottler',
 'Orbeetle',
 'Fomantis',
 'Lurantis',
 'Applin',
 'Flapple',
 'Appletun',
 'Fletchling',
 'Fletchinder',
 'Talonflame',
 'Shinx',
 'Luxio',
 'Luxray',
 'Klefki',
 'Pawniard',
 'Bisharp',
 'Abra',
 'Kadabra',
 'Alakazam',
 'Ralts',
 'Kirlia',
 'Gardevoir',
 'Gallade',
 'Kra

#### Restricted

In [150]:
restricted_VGC2022 = list(
    pokemon_i_df[pokemon_i_df['#'].isin(national_restricted)
                 & pd.isnull(pokemon_i_df['Variant'])]['Name'])
len(restricted_VGC2022)

19

In [161]:
restricted_VGC2022

['Mewtwo',
 'Lugia',
 'Ho-oh',
 'Kyogre',
 'Groudon',
 'Rayquaza',
 'Dialga',
 'Palkia',
 'Reshiram',
 'Zekrom',
 'Kyurem',
 'Xerneas',
 'Yveltal',
 'Cosmog',
 'Cosmoem',
 'Solgaleo',
 'Lunala',
 'Necrozma',
 'Calyrex']

##### Banned

In [151]:
pokemon_i_df[pokemon_i_df['Name'] == 'Mega Charizard X']

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
9,6,Mega Charizard X,Charizard,Mega X,6,Common,Charizard,False,Fire,Dragon,634,78,130,111,130,85,100,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...


In [152]:
banned = ['Mew',
 'Celebi',
 'Jirachi',
 'Victini',
 'Keldeo',
 'Genesect',
 'Diancie',
 'Volcanion',
 'Magearna',
 'Marshadow',
 'Zeraora',
 'Meltan',
 'Melmetal',
 'Zarude']

In [163]:
banned_VGC2022 = list(
    pokemon_i_df[pokemon_i_df['Species'].isin(banned)
                 | pokemon_i_df['Variant'].str.contains('Mega')]['Name'])

- Questi sopra sono i nomi delle specie !
- Bisogna anche aggiungere tutte le mega

In [164]:
banned_VGC2022

['Mega Venusaur',
 'Mega Charizard Y',
 'Mega Charizard X',
 'Mega Blastoise',
 'Mega Beedrill',
 'Mega Pidgeot',
 'Mega Alakazam',
 'Mega Slowbro',
 'Mega Gengar',
 'Mega Kangaskhan',
 'Mega Pinsir',
 'Mega Gyarados',
 'Mega Aerodactyl',
 'Mega Mewtwo Y',
 'Mega Mewtwo X',
 'Mew',
 'Mega Ampharos',
 'Mega Steelix',
 'Mega Scizor',
 'Mega Heracross',
 'Mega Houndoom',
 'Mega Tyranitar',
 'Celebi',
 'Mega Sceptile',
 'Mega Blaziken',
 'Mega Swampert',
 'Mega Gardevoir',
 'Mega Sableye',
 'Mega Mawile',
 'Mega Aggron',
 'Mega Medicham',
 'Mega Manectric',
 'Mega Sharpedo',
 'Mega Camerupt',
 'Mega Altaria',
 'Mega Banette',
 'Mega Absol',
 'Mega Glalie',
 'Mega Salamence',
 'Mega Metagross',
 'Mega Latias',
 'Mega Latios',
 'Mega Rayquaza',
 'Jirachi',
 'Mega Lopunny',
 'Mega Garchomp',
 'Mega Lucario',
 'Mega Abomasnow',
 'Mega Gallade',
 'Victini',
 'Mega Audino',
 'Keldeo Resolute Form',
 'Keldeo Ordinary Form',
 'Genesect',
 'Mega Diancie',
 'Diancie',
 'Volcanion',
 'Magearna',
 'Ma

##### Gigantamax allowed

In [154]:
gigantamax_allowed_list

['Venusaur',
 'Charizard',
 'Blastoise',
 'Butterfree',
 'Pikachu',
 'Meowth',
 'Machamp',
 'Gengar',
 'Kingler',
 'Lapras',
 'Eevee',
 'Snorlax',
 'Garbodor',
 'Rillaboom',
 'Cinderace',
 'Inteleon',
 'Corviknight',
 'Orbeetle',
 'Drednaw',
 'Coalossal',
 'Flapple',
 'Appletun',
 'Sandaconda',
 'Toxtricity(both forms)',
 'Centiskorch',
 'Hatterene',
 'Grimmsnarl',
 'Alcremie(all forms)',
 'Copperajah',
 'Duraludon',
 'Single Strike Style Urshifu',
 'Rapid Strike Style Urshifu']

In [157]:
gigantamax_allowed_list = ['Venusaur',
 'Charizard',
 'Blastoise',
 'Butterfree',
 'Pikachu',
 'Meowth',
 'Machamp',
 'Gengar',
 'Kingler',
 'Lapras',
 'Eevee',
 'Snorlax',
 'Garbodor',
 'Rillaboom',
 'Cinderace',
 'Inteleon',
 'Corviknight',
 'Orbeetle',
 'Drednaw',
 'Coalossal',
 'Flapple',
 'Appletun',
 'Sandaconda',
 'Toxtricity',
 'Centiskorch',
 'Hatterene',
 'Grimmsnarl',
 'Alcremie',
 'Copperajah',
 'Duraludon',
 'Urshifu (Single Strike)',
 'Urshifu (Rapid Strike)']

In [172]:
gigantamax_allowed_VGC2022 = list(
    pokemon_i_df[pokemon_i_df['Species'].isin(gigantamax_allowed_list)
                 & pokemon_i_df['Variant'].str.contains('Gigantamax')]['Name'])

In [173]:
gigantamax_allowed_VGC2022

['Venusaur Gigantamax',
 'Charizard Gigantamax',
 'Blastoise Gigantamax',
 'Butterfree Gigantamax',
 'Pikachu Gigantamax',
 'Meowth Gigantamax',
 'Machamp Gigantamax',
 'Gengar Gigantamax',
 'Kingler Gigantamax',
 'Lapras Gigantamax',
 'Eevee Gigantamax',
 'Snorlax Gigantamax',
 'Garbodor Gigantamax',
 'Rillaboom Gigantamax',
 'Cinderace Gigantamax',
 'Inteleon Gigantamax',
 'Corviknight Gigantamax',
 'Orbeetle Gigantamax',
 'Drednaw Gigantamax',
 'Coalossal Gigantamax',
 'Flapple Gigantamax',
 'Appletun Gigantamax',
 'Sandaconda Gigantamax',
 'Centiskorch Gigantamax',
 'Hatterene Gigantamax',
 'Grimmsnarl Gigantamax',
 'Alcremie Gigantamax',
 'Copperajah Gigantamax',
 'Duraludon Gigantamax',
 'Urshifu (Single Strike) Gigantamax',
 'Urshifu (Rapid Strike) Gigantamax']

### Creazione colonna `VGC2022_rules Competitivo`

In [167]:
pokemon_i_df['VGC2022_rules'] = pd.NA

In [170]:
pokemon_i_df

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url,VGC2022_rules
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...,NaN
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...,NaN
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...,NaN
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...,NaN
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,896,Glastrier,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...,NaN
1094,897,Spectrier,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...,NaN
1095,898,Calyrex,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...,NaN
1096,898,Calyrex Shadow Rider,Calyrex,Shadow Rider,8,Legendary,<NA>,False,Psychic,Ghost,680,100,85,80,165,100,150,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...,NaN


In [179]:
pokemon_i_df['VGC2022_rules'].loc[pokemon_i_df['Name'].isin(permitted_VGC2022)] = 'Permitted'
pokemon_i_df['VGC2022_rules'].loc[pokemon_i_df['Name'].isin(restricted_VGC2022)] = 'Restricted (players can include two Restricted Pokémon in their team)'
pokemon_i_df['VGC2022_rules'].loc[pokemon_i_df['Name'].isin(banned_VGC2022)] = 'Banned'
pokemon_i_df['VGC2022_rules'].loc[pokemon_i_df['Name'].isin(gigantamax_allowed_VGC2022)] = 'Gigantamax Allowed'

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [183]:
pokemon_i_df.head(10)

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url,VGC2022_rules
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...,Permitted
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...,Permitted
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...,Permitted
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...,Banned
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...,Gigantamax Allowed
5,4,Charmander,Charmander,<NA>,1,Common,<NA>,False,Fire,<NA>,309,39,52,43,60,50,65,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...,Permitted
6,5,Charmeleon,Charmeleon,<NA>,1,Common,Charmander,False,Fire,<NA>,405,58,64,58,80,65,80,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...,Permitted
7,6,Mega Charizard Y,Charizard,Mega Y,6,Common,Charizard,False,Fire,Flying,634,78,104,78,159,115,100,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...,Banned
8,6,Charizard Gigantamax,Charizard,Gigantamax,8,<NA>,Charizard,<NA>,Fire,Flying,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...,Gigantamax Allowed
9,6,Mega Charizard X,Charizard,Mega X,6,Common,Charizard,False,Fire,Dragon,634,78,130,111,130,85,100,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...,Banned


## Competitivo: integrazione e tabelle ponte (sistemazione `Items` e `Natures`, integrazione `pokemon`, creazione `common spread`)

### Dataset entità: sistemazione

#### Items: sistemazione dataset

In [201]:
items_i_df = pd.read_csv('./Dataset integrati/item_integrati.csv', index_col=0)

In [202]:
items_i_df = items_i_df.replace(['[]'], np.nan)
items_i_df = items_i_df.convert_dtypes()
items_i_df.rename(
    columns={
        'Effect_y': 'Effect',
        'Effect_x': 'Effect (alt.)',
        'Effect_y': 'Effect',
        'Category_y': 'General Category',
        'Category_x': 'Specific Category',
        'Attributes':'Usage Attributes',
        'sprite_url':'image_url'
    }, inplace=True)

In [203]:
items_i_df.head()

,Name,Effect (alt.),Effect,Usage Attributes,Specific Category,General Category,image_url
0,Ability Capsule,Switches a Pokémon between its two possible (n...,A capsule that allows a Pokémon with two Abili...,<NA>,Vitamins,Hold items,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,<NA>,Changes a Pokémon's ability to its Hidden Abil...,<NA>,Vitamins,General items,https://img.pokemondb.net/s.png
2,Ability Urge,Forcibly activates a friendly Pokémon's ability.,"When used, it activates the Ability of an ally...",<NA>,Miracle Shooter,Battle items,https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Held: Allows Abomasnow to Mega Evolve into Meg...,Enables Abomasnow to Mega Evolve during battle.,<NA>,Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Held: Allows Absol to Mega Evolve into Mega Ab...,Enables Absol to Mega Evolve during battle.,<NA>,Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/absoli...


#### natures: importazione dataset

In [204]:
natures_i_df = pd.read_csv('./Scraping_datasets/pokemondb/natures.csv', index_col=0)

In [210]:
natures_i_df.head(5)

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy


#### Pokemon (il dataset ottenuto dalle precedenti integrazioni)

In [211]:
pokemon_i_df.head()

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,image_url,VGC2022_rules
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/sword-shield...,Permitted
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/sword-shield...,Permitted
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/sword-shield...,Permitted
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/sword-shield...,Banned
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...,Gigantamax Allowed


#### Moves (il dataset ottenuto dalle precedenti integrazioni) -> Sistemazione

In [208]:
moves_i_df.drop(axis=1, labels='Learned_by', inplace=True)
moves_i_df.head()

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%)
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10
3,Acid Armor,1,Poison,<NA>,<NA>,20,Status,Sharply raises user's Defense.,<NA>
4,Acid Spray,5,Poison,40,100,20,Special,Sharply lowers opponent's Special Defense.,100
...,...,...,...,...,...,...,...,...,...
816,G-Max Finale,8,Fairy,<NA>,∞,5,<NA>,Heals the user and its allies by 1/6 their max...,<NA>
817,G-Max Steelsurge,8,Steel,<NA>,∞,5,<NA>,Scatters sharp spikes around the field Works l...,<NA>
818,G-Max Depletion,8,Dragon,<NA>,∞,5,<NA>,Takes away 2 PP from the last move the target ...,<NA>
819,G-Max One Blow,8,Dark,<NA>,∞,5,<NA>,Hits the target even if it is protected by a p...,<NA>


#### Ability

In [212]:
abilities_i_df.head()

,Name,Generation,Description
0,Adaptability,4,Powers up moves of the same type.
1,Aerilate,6,Turns Normal-type moves into Flying-type moves.
2,Aftermath,4,Damages the attacker landing the finishing hit.
3,Air Lock,3,Eliminates the effects of weather.
4,Analytic,5,Boosts move power when the Pokémon moves last.


### Dataset competitivo: importazione e sistemazione

In [389]:
usage_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/usage_S12VGC.csv', index_col=0)
usage_S12VGC.head()

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,Zacian-Crowned,1088,65,1
1,Incineroar,1124,59,2
2,Kyogre,744,35,3
3,Grimmsnarl,577,30,4
4,Regieleki,603,29,5


In [215]:
teammates_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/teammates_S12VGC.csv', index_col=0)
teammates_S12VGC.head()

,Pokemon,Teammate,Use_Percentage (%)
0,Zacian-Crowned,Incineroar,54.189%
1,Zacian-Crowned,Kyogre,31.467%
2,Zacian-Crowned,Grimmsnarl,31.148%
3,Zacian-Crowned,Rillaboom,29.083%
4,Zacian-Crowned,Gastrodon,24.347%


In [216]:
moves_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/moves_S12VGC.csv', index_col=0)
moves_S12VGC.head()

,Pokemon,Move,Type,Use_Percentage (%)
0,Zacian-Crowned,Behemoth Blade,steel,99.996%
1,Zacian-Crowned,Protect,normal,98.672%
2,Zacian-Crowned,Sacred Sword,fighting,84.154%
3,Zacian-Crowned,Play Rough,fairy,64.487%
4,Zacian-Crowned,Substitute,normal,30.274%


In [217]:
items_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/items_S12VGC.csv', index_col=0)
items_S12VGC.head()

,Pokemon,Item,Use_Percentage (%)
0,Zacian-Crowned,Rusted Sword,100.000%
1,Incineroar,Shuca Berry,36.082%
2,Incineroar,Safety Goggles,23.390%
3,Incineroar,Sitrus Berry,14.031%
4,Incineroar,Focus Sash,8.994%


In [218]:
EV_spread_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/EV_spread_S12VGC.csv', index_col=0)
EV_spread_S12VGC.head()

,Pokemon,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%)
0,Zacian-Crowned,Jolly,0/252/0/0/4/252,10.388%
1,Zacian-Crowned,Adamant,252/28/4/0/12/212,4.504%
2,Zacian-Crowned,Jolly,4/252/0/0/0/252,4.419%
3,Zacian-Crowned,Adamant,188/164/4/0/4/148,2.891%
4,Zacian-Crowned,Jolly,0/252/4/0/0/252,2.456%


In [219]:
basestat_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/basestat_S12VGC.csv', index_col=0)
basestat_S12VGC.head()

,Pokemon,HP,Atk,Def,SpA,SpD,Spe
0,Zacian-Crowned,92,170,115,80,115,148
1,Grimmsnarl,95,120,65,95,75,60
2,Regieleki,80,100,50,100,50,200
3,Rillaboom,100,125,90,60,70,85
4,Calyrex-Shadow,100,85,80,165,100,150


In [220]:
abilities_S12VGC = pd.read_csv('./Scraping_datasets/pikalytics/abilities_S12VGC.csv', index_col=0)
abilities_S12VGC.head()

,Pokemon,Ability,Use_Percentage (%)
0,Zacian-Crowned,Intrepid Sword,100.000%
1,Incineroar,Intimidate,99.927%
2,Incineroar,Blaze,0.073%
3,Kyogre,Drizzle,100.000%
4,Grimmsnarl,Prankster,99.959%


### TABELLE PONTE per il Matching Entità con nomi entità competitivo (pokemon, items, moves, abilities)

L'idea è quella di creare delle tabelle ponte che permettano di associare ad ogni nome competitivo il suo corrispettivo nei nomi dei dataset integrati

#### Pokemon (uso usage ma andrà bene per tutti i dataset con i pokemon)

Pulizia stringhe

In [422]:
# Elimino trattini tra i nomi
usage_S12VGC['Pokemon'] = rl.preprocessing.clean(
    usage_S12VGC['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
usage_S12VGC['Pokemon'] = usage_S12VGC['Pokemon'].str.title()

In [423]:
usage_S12VGC.head(10)

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,Zacian Crowned,1088,65,1
1,Incineroar,1124,59,2
2,Kyogre,744,35,3
3,Grimmsnarl,577,30,4
4,Regieleki,603,29,5
5,Rillaboom,482,27,6
6,Groudon,501,23,7
7,Calyrex Shadow,363,22,8
8,Thundurus,352,21,9
9,Landorus Therian,369,20,10


Faccio una serie di check:
- A quanto pare il matching serve farlo solo sui nomi dei pokemon (tutte e 788 le mosse della tabella ponte si trovano identicamente del dataset delle mosse)
- Mancano delle mosse (le G-Max) che però hanno la loro relazionbe specifica

In [424]:
pokemon_i_df.shape

(1098, 19)

Un certo numero di pokemon sono già dentro

In [425]:
sum(pokemon_i_df['Name'].isin(usage_S12VGC['Pokemon']))

283

In [426]:
len(usage_S12VGC['Pokemon'].unique())

342

Ci sono solo 342 pokemon dentro

##### Record Linkage

In [427]:
pokemon_i_df.head(2)

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,image_url,VGC2022_rules
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/sword-shield...,Permitted
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/sword-shield...,Permitted


In [428]:
usage_S12VGC

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,Zacian Crowned,1088,65,1
1,Incineroar,1124,59,2
2,Kyogre,744,35,3
3,Grimmsnarl,577,30,4
4,Regieleki,603,29,5
...,...,...,...,...
337,Steelix,-,-,-
338,Noctowl,-,-,-
339,Corsola,-,-,-
340,Mime Jr,-,-,-


Essendo il dataset troppo grande procedo elaborando i soli valori unici, per poi generare una tabella ponte e ricongiungermi ai 781982 record iniziali

In [429]:
ponte_pokemon_COMPETITIVE_unique = pd.DataFrame(usage_S12VGC['Pokemon'].unique(), columns=['Pokemon'])

In [430]:
ponte_pokemon_COMPETITIVE_unique

,Pokemon
0,Zacian Crowned
1,Incineroar
2,Kyogre
3,Grimmsnarl
4,Regieleki
...,...
337,Steelix
338,Noctowl
339,Corsola
340,Mime Jr


Setup

In [431]:
indexer = rl.Index()
indexer.full()

<Index>

In [432]:
candidates = indexer.index(ponte_pokemon_COMPETITIVE_unique, pokemon_i_df)
len(candidates)

375516

Comparing del nome

In [433]:
compare = rl.Compare()
compare.string('Pokemon', 'Name', threshold=0.75, label='Name', method='qgram')
features = compare.compute(candidates, ponte_pokemon_COMPETITIVE_unique,
                           pokemon_i_df)

Come atteso matchano tutte

In [434]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0       323
0.0    375193
dtype: int64

Estraggo i match potenziali

In [435]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

In [436]:
potential_matches

,level_0,level_1,Score
0,1,886,1.0
1,2,479,1.0
2,3,1046,1.0
3,4,1091,1.0
4,5,984,1.0
...,...,...,...
318,338,226,1.0
319,339,291,1.0
320,339,1049,1.0
321,340,546,1.0


Mergio i due datasets

In [437]:
ponte_pokemon_COMPETITIVE_unique_ = pd.merge(potential_matches, ponte_pokemon_COMPETITIVE_unique, left_on="level_0", right_index=True)
ponte_pokemon_COMPETITIVE_unique_ = pd.merge(ponte_pokemon_COMPETITIVE_unique_, pokemon_i_df[['Name', 'Variant']], left_on="level_1", right_index=True)

In [438]:
pd.set_option('display.max_rows', 30)
ponte_pokemon_COMPETITIVE_unique_

,level_0,level_1,Score,Pokemon,Name,Variant
0,1,886,1.0,Incineroar,Incineroar,<NA>
1,2,479,1.0,Kyogre,Kyogre,<NA>
2,3,1046,1.0,Grimmsnarl,Grimmsnarl,<NA>
3,4,1091,1.0,Regieleki,Regieleki,<NA>
4,5,984,1.0,Rillaboom,Rillaboom,<NA>
...,...,...,...,...,...,...
316,336,43,1.0,Sandslash,Sandslash,<NA>
317,337,272,1.0,Steelix,Steelix,<NA>
318,338,226,1.0,Noctowl,Noctowl,<NA>
321,340,546,1.0,Mime Jr,Mime Jr.,Jr.


Elimino le righe errate dopo averle controllate

In [439]:
ponte_pokemon_COMPETITIVE_unique_ = ponte_pokemon_COMPETITIVE_unique_.drop([12,134, 173, 203, 275, 320, 290])

Tengo le sole righe utili

In [440]:
ponte_pokemon_COMPETITIVE_unique_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Variant'], inplace=True)
ponte_pokemon_COMPETITIVE_unique_

,Pokemon,Name
0,Incineroar,Incineroar
1,Kyogre,Kyogre
2,Grimmsnarl,Grimmsnarl
3,Regieleki,Regieleki
4,Rillaboom,Rillaboom
...,...,...
316,Sandslash,Sandslash
317,Steelix,Steelix
318,Noctowl,Noctowl
321,Mime Jr,Mime Jr.


**Ri-matcho le righe non matchate con un livello più basso**

In [441]:
ponte_pokemon_COMPETITIVE_unique_aggiuntivi = pd.DataFrame(usage_S12VGC[
    ~usage_S12VGC['Pokemon'].isin(ponte_pokemon_COMPETITIVE_unique_['Pokemon']
                                  )]['Pokemon'], columns=['Pokemon'])

In [442]:
indexer = rl.Index()
indexer.full()
candidates = indexer.index(ponte_pokemon_COMPETITIVE_unique_aggiuntivi, pokemon_i_df)
compare = rl.Compare()
compare.string('Pokemon', 'Name', threshold=0.50, label='Name', method='qgram')
features = compare.compute(candidates, ponte_pokemon_COMPETITIVE_unique_aggiuntivi,
                           pokemon_i_df)
#features.sum(axis=1).value_counts().sort_index(ascending=False)
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(
    columns={'Name': 'Score'})
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_ = pd.merge(potential_matches,
                                             ponte_pokemon_COMPETITIVE_unique_aggiuntivi,
                                             left_on="level_0",
                                             right_index=True)
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_ = pd.merge(ponte_pokemon_COMPETITIVE_unique_aggiuntivi_,
                                             pokemon_i_df[['Name', 'Variant']],
                                             left_on="level_1",
                                             right_index=True)
#ponte_pokemon_COMPETITIVE_unique_aggiuntivi_
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Variant'], inplace=True)
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_

,Pokemon,Name
0,Zacian Crowned,Zacian Crowned Sword
1,Calyrex Shadow,Calyrex
7,Calyrex Ice,Calyrex
2,Calyrex Shadow,Calyrex Shadow Rider
3,Landorus Therian,Thundurus Therian Forme
4,Landorus Therian,Landorus Therian Forme
5,Indeedee F,Indeedee Female
10,Indeedee,Indeedee Female
6,Indeedee F,Indeedee Male
11,Indeedee,Indeedee Male


Elimino le righe non giuste e correggo

In [443]:
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_ = ponte_pokemon_COMPETITIVE_unique_aggiuntivi_.drop(
    [1, 7,3,10,6,12,14,15,17,20,24,25,26,28])

In [444]:
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_

,Pokemon,Name
0,Zacian Crowned,Zacian Crowned Sword
2,Calyrex Shadow,Calyrex Shadow Rider
4,Landorus Therian,Landorus Therian Forme
5,Indeedee F,Indeedee Female
11,Indeedee,Indeedee Male
8,Calyrex Ice,Calyrex Ice Rider
9,Ho Oh,Ho-oh
13,Zamazenta Crowned,Zamazenta Crowned Shield
16,Darmanitan Galar,Darmanitan Zen Mode
21,Darmanitan,Darmanitan Zen Mode


In [445]:
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_['Name'].loc[
    ponte_pokemon_COMPETITIVE_unique_aggiuntivi_['Pokemon'] ==
    'Darmanitan Galar'] = 'Darmanitan Galarian Standard Mode'
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_['Name'].loc[
    ponte_pokemon_COMPETITIVE_unique_aggiuntivi_['Pokemon'] ==
    'Darmanitan'] = 'Darmanitan Standard Mode'
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_['Name'].loc[
    ponte_pokemon_COMPETITIVE_unique_aggiuntivi_['Pokemon'] ==
    'Toxtricity'] = 'Toxtricity Low Key Form'
ponte_pokemon_COMPETITIVE_unique_aggiuntivi_

,Pokemon,Name
0,Zacian Crowned,Zacian Crowned Sword
2,Calyrex Shadow,Calyrex Shadow Rider
4,Landorus Therian,Landorus Therian Forme
5,Indeedee F,Indeedee Female
11,Indeedee,Indeedee Male
8,Calyrex Ice,Calyrex Ice Rider
9,Ho Oh,Ho-oh
13,Zamazenta Crowned,Zamazenta Crowned Shield
16,Darmanitan Galar,Darmanitan Galarian Standard Mode
21,Darmanitan,Darmanitan Standard Mode


**Concateno i dataset dei matchati con i nuovi matchati aggiuntivi e ottengo il dataset finale**

In [457]:
frame = [ponte_pokemon_COMPETITIVE_unique_, ponte_pokemon_COMPETITIVE_unique_aggiuntivi_]
ponte_pokemon_COMPETITIVE = pd.concat(frame)
ponte_pokemon_COMPETITIVE.reset_index(drop=True, inplace=True)
ponte_pokemon_COMPETITIVE

,Pokemon,Name
0,Incineroar,Incineroar
1,Kyogre,Kyogre
2,Grimmsnarl,Grimmsnarl
3,Regieleki,Regieleki
4,Rillaboom,Rillaboom
5,Groudon,Groudon
6,Gastrodon,Gastrodon
7,Charizard,Charizard
8,Amoonguss,Amoonguss
9,Whimsicott,Whimsicott


In [458]:
ponte_pokemon_COMPETITIVE.shape

(331, 2)

**Ne manco ancora alcuni, li aggiungo manualmente (controllo dallo sprite di pikalitics)**

In [462]:
#usage_S12VGC[~usage_S12VGC['Pokemon'].isin(usage_S12VGC_['Pokemon'])]

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
8,Thundurus,352,21,9
19,Tornadus,191,8,20
26,Urshifu,125,4,27
62,Zygarde,42,1,63
77,Landorus,23,0,78
147,Aegislash,10,0,148
189,Giratina,9,0,190
245,Eiscue,-,-,-
272,Lycanroc Dusk,-,-,-
283,Morpeko,-,-,-


In [477]:
competitive_manual = np.array([['Thundurus', 'Thundurus Incarnate Forme'],
                               ['Tornadus', 'Tornadus Incarnate Forme'],
                               ['Urshifu', 'Urshifu Single Strike Style'],
                               ['Zygarde', 'Zygarde Complete Forme'],
                               ['Landorus', 'Landorus Incarnate Forme'],
                               ['Aegislash', 'Aegislash Shield Forme'],
                               ['Giratina', 'Giratina Altered Forme'],
                               ['Eiscue', 'Eiscue Ice Face'],
                               ['Lycanroc Dusk', 'Lycanroc Dusk Form'],
                               ['Morpeko', 'Morpeko Full Belly Mode'],
                               ['Lycanroc', 'Lycanroc Midday Form']])
ponte_pokemon_COMPETITIVE_manual = pd.DataFrame(competitive_manual, columns=['Pokemon', 'Name'])

In [480]:
frame = [ponte_pokemon_COMPETITIVE, ponte_pokemon_COMPETITIVE_manual]
ponte_pokemon_COMPETITIVE = pd.concat(frame)
ponte_pokemon_COMPETITIVE.reset_index(drop=True, inplace=True)
ponte_pokemon_COMPETITIVE

,Pokemon,Name
0,Incineroar,Incineroar
1,Kyogre,Kyogre
2,Grimmsnarl,Grimmsnarl
3,Regieleki,Regieleki
4,Rillaboom,Rillaboom
...,...,...
337,Giratina,Giratina Altered Forme
338,Eiscue,Eiscue Ice Face
339,Lycanroc Dusk,Lycanroc Dusk Form
340,Morpeko,Morpeko Full Belly Mode


#### Abilities

In [306]:
len(abilities_S12VGC['Ability'].unique())

225

Le abilità sembrano matchare bene, solo una non è presente

In [314]:
abilities_S12VGC[~abilities_S12VGC['Ability'].isin(abilities_i_df['Name'])]

,Pokemon,Ability,Use_Percentage (%)
442,Silvally,RKS System,100.000%


In [310]:
sum(abilities_i_df['Name'].isin(abilities_S12VGC['Ability']))

224

Vado a cambiare quella non presente per farla matchare

In [315]:
abilities_S12VGC.loc[
    abilities_S12VGC['Ability'] ==
    'RKS System'] = 'Rks System'

Risolto

In [316]:
abilities_S12VGC[~abilities_S12VGC['Ability'].isin(abilities_i_df['Name'])]

,Pokemon,Ability,Use_Percentage (%)


#### Items

In [318]:
items_S12VGC.head()

,Pokemon,Item,Use_Percentage (%)
0,Zacian-Crowned,Rusted Sword,100.000%
1,Incineroar,Shuca Berry,36.082%
2,Incineroar,Safety Goggles,23.390%
3,Incineroar,Sitrus Berry,14.031%
4,Incineroar,Focus Sash,8.994%


In [317]:
items_i_df.head()

,Name,Effect (alt.),Effect,Usage Attributes,Specific Category,General Category,image_url
0,Ability Capsule,Switches a Pokémon between its two possible (n...,A capsule that allows a Pokémon with two Abili...,<NA>,Vitamins,Hold items,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,<NA>,Changes a Pokémon's ability to its Hidden Abil...,<NA>,Vitamins,General items,https://img.pokemondb.net/s.png
2,Ability Urge,Forcibly activates a friendly Pokémon's ability.,"When used, it activates the Ability of an ally...",<NA>,Miracle Shooter,Battle items,https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Held: Allows Abomasnow to Mega Evolve into Meg...,Enables Abomasnow to Mega Evolve during battle.,<NA>,Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Held: Allows Absol to Mega Evolve into Mega Ab...,Enables Absol to Mega Evolve during battle.,<NA>,Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/absoli...


In [319]:
len(items_S12VGC['Item'].unique())

115

In [320]:
sum(items_i_df['Name'].isin(items_S12VGC['Item']))

108

In [322]:
items_S12VGC[~items_S12VGC['Item'].isin(items_i_df['Name'])]

,Pokemon,Item,Use_Percentage (%)
0,Zacian-Crowned,Rusted Sword,100.000%
10,Incineroar,Other,4.928%
19,Kyogre,Other,3.326%
25,Grimmsnarl,Other,4.740%
31,Regieleki,Other,4.582%
38,Rillaboom,Other,3.811%
46,Groudon,Other,4.760%
51,Calyrex-Shadow,Other,1.108%
59,Thundurus,Other,3.816%
66,Landorus-Therian,Other,2.786%


Anche questi vanno bene così come sono

#### Moves

In [324]:
moves_S12VGC.head()

,Pokemon,Move,Type,Use_Percentage (%)
0,Zacian-Crowned,Behemoth Blade,steel,99.996%
1,Zacian-Crowned,Protect,normal,98.672%
2,Zacian-Crowned,Sacred Sword,fighting,84.154%
3,Zacian-Crowned,Play Rough,fairy,64.487%
4,Zacian-Crowned,Substitute,normal,30.274%


In [325]:
moves_i_df.head()

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%)
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10
3,Acid Armor,1,Poison,<NA>,<NA>,20,Status,Sharply raises user's Defense.,<NA>
4,Acid Spray,5,Poison,40,100,20,Special,Sharply lowers opponent's Special Defense.,100


In [326]:
len(moves_S12VGC['Move'].unique())

421

Ci sono circa 16 cose che non vanno, procedo con il **matching**

In [334]:
sum(moves_i_df['Name'].isin(moves_S12VGC['Move']))

404

In [345]:
# Elimino trattini tra i nomi
moves_S12VGC['Move'] = rl.preprocessing.clean(
    moves_S12VGC['Move'])
# Rendo Maiuscola la prima lettera di ogni parola
moves_S12VGC['Move'] = moves_S12VGC['Move'].str.title()

In [361]:
ponte_move_COMPETITIVE_unique = pd.DataFrame(moves_S12VGC['Move'].unique(), columns=['Move'])
ponte_move_COMPETITIVE_unique.shape

(421, 1)

In [362]:
indexer = rl.Index()
indexer.full()
candidates = indexer.index(ponte_move_COMPETITIVE_unique, moves_i_df)
compare = rl.Compare()
compare.string('Move', 'Name', threshold=0.80, label='Name', method='qgram')
features = compare.compute(candidates, ponte_move_COMPETITIVE_unique,
                           moves_i_df)
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0       422
0.0    345219
dtype: int64

In [363]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(
    columns={'Name': 'Score'})
ponte_move_COMPETITIVE_unique_ = pd.merge(potential_matches,
                                             ponte_move_COMPETITIVE_unique,
                                             left_on="level_0",
                                             right_index=True)
ponte_move_COMPETITIVE_unique_ = pd.merge(ponte_move_COMPETITIVE_unique_,
                                             moves_i_df[['Name', 'Type']],
                                             left_on="level_1",
                                             right_index=True)
#ponte_pokemon_COMPETITIVE_unique_aggiuntivi_
ponte_move_COMPETITIVE_unique_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Type'], inplace=True)
ponte_move_COMPETITIVE_unique_.head()

,Move,Name
0,Behemoth Blade,Behemoth Blade
1,Protect,Protect
2,Sacred Sword,Sacred Sword
3,Play Rough,Play Rough
4,Substitute,Substitute


C'è un match in più rispetto a quelli attesi, controllo i valori non uguali per vedere quale è sbagliato

In [364]:
ponte_move_COMPETITIVE_unique_[ponte_move_COMPETITIVE_unique_['Move'] != ponte_move_COMPETITIVE_unique_['Name']][['Move', 'Name']]

,Move,Name
13,Water Spout,Water Sport
181,Pyro Ball,Gyro Ball
124,Gyro Ball,Pyro Ball
162,Natures Madness,Nature'S Madness
307,Kings Shield,King'S Shield
323,Forests Curse,Forest'S Curse


In [365]:
ponte_move_COMPETITIVE = ponte_move_COMPETITIVE_unique_.drop([13,181,124])

Ne mancano ancora 2, proceso con il controllo

In [372]:
moves_S12VGC[~moves_S12VGC['Move'].isin(moves_i_df['Name'])]['Move'].unique()

array(['Other', nan, 'Natures Madness', 'Kings Shield', 'Forests Curse'],
      dtype=object)

Sono other e nan (posso quindi considerare avvenuto il matching perfettamente)

#### Natures

In [373]:
natures_i_df.head()

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy


In [374]:
EV_spread_S12VGC.head()

,Pokemon,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%)
0,Zacian-Crowned,Jolly,0/252/0/0/4/252,10.388%
1,Zacian-Crowned,Adamant,252/28/4/0/12/212,4.504%
2,Zacian-Crowned,Jolly,4/252/0/0/0/252,4.419%
3,Zacian-Crowned,Adamant,188/164/4/0/4/148,2.891%
4,Zacian-Crowned,Jolly,0/252/4/0/0/252,2.456%


In [376]:
len(natures_i_df)

25

In [375]:
len(EV_spread_S12VGC['Nature'].unique())

25

Ci sono tutte, quindi non serve il ponte

#### TABELLE PONTE OTTENUTE

- ponte_pokemon_COMPETITIVE
- abilities_S12VGC['Ability'] (va già bene)
- items_i_df['Name'] (va già bene)
- ponte_move_COMPETITIVE
- natures_i_df['Natures']

### Salvataggio `df_items`, `df_moves`, `df_natures`

In [377]:
natures_i_df.to_csv(
    './Dataset integrati 2.0/df_natures.csv',
    index=False)

moves_i_df.to_csv(
    './Dataset integrati 2.0/df_moves.csv',
    index=False)

items_i_df.to_csv(
    './Dataset integrati 2.0/df_items.csv',
    index=False)

### POKEMON: integrazione usage_S12VGC e salvataggio

Vado a combinare le tabelle ponte ottenute con usage_S12VGC per poi conbinare con pokemon_if_df

In [446]:
pokemon_i_df.shape # il numero di righe alla fine deve essere uguale

(1098, 19)

In [447]:
usage_S12VGC.shape

(342, 4)

In [448]:
usage_S12VGC.head()

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,Zacian Crowned,1088,65,1
1,Incineroar,1124,59,2
2,Kyogre,744,35,3
3,Grimmsnarl,577,30,4
4,Regieleki,603,29,5


In [449]:
ponte_pokemon_COMPETITIVE.head()

,Pokemon,Name
0,Incineroar,Incineroar
1,Kyogre,Kyogre
2,Grimmsnarl,Grimmsnarl
3,Regieleki,Regieleki
4,Rillaboom,Rillaboom


In [491]:
usage_S12VGC_ = pd.merge(usage_S12VGC, ponte_pokemon_COMPETITIVE, on = 'Pokemon').drop(axis=1, labels='Pokemon')

In [492]:
usage_S12VGC_

,Monthly Usage (k),Usage Percent (%),Monthly Rank,Name
0,1088,65,1,Zacian Crowned Sword
1,1124,59,2,Incineroar
2,744,35,3,Kyogre
3,577,30,4,Grimmsnarl
4,603,29,5,Regieleki
...,...,...,...,...
337,-,-,-,Steelix
338,-,-,-,Noctowl
339,-,-,-,Corsola
340,-,-,-,Mime Jr.


Combino con pokemon_i_df

In [493]:
pokemon_i_df_comp = pd.merge(pokemon_i_df, usage_S12VGC_, on = 'Name', how = 'left')

**Salvataggio**

In [495]:
pokemon_i_df_comp.columns

Index(['#', 'Name', 'Species', 'Variant', 'Generation', 'Rarity',
       'Evolves_from', 'Has_gender_diff', 'Type1', 'Type2', 'Total', 'HP',
       'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'image_url',
       'VGC2022_rules', 'Monthly Usage (k)', 'Usage Percent (%)',
       'Monthly Rank'],
      dtype='object')

In [496]:
pokemon_i_df_comp.head()

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,...,Attack,Defense,Sp. Atk,Sp. Def,Speed,image_url,VGC2022_rules,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,...,49,49,65,65,45,https://img.pokemondb.net/sprites/sword-shield...,Permitted,NaN,NaN,NaN
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,...,62,63,80,80,60,https://img.pokemondb.net/sprites/sword-shield...,Permitted,NaN,NaN,NaN
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,...,82,83,100,100,80,https://img.pokemondb.net/sprites/sword-shield...,Permitted,204,7,24
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,...,100,123,122,120,80,https://img.pokemondb.net/sprites/sword-shield...,Banned,NaN,NaN,NaN
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,...,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...,Gigantamax Allowed,NaN,NaN,NaN


In [497]:
pokemon_i_df_comp.to_csv(
    './Dataset integrati 2.0/df_pokemon.csv',
    index=False)

### Integrazione e salvataggio TABELLE PONTE COMPETITIVO: `USED_IN_TEAM_WITH`, `USED_WITH_ITEM`, `USED_WITH_MOVE`, `COMMON_SPREAD` e `USED_WITH_ABILITY`

#### `USED_IN_TEAM_WITH`

Pulizia stringhe

In [541]:
#teammates_S12VGC[~teammates_S12VGC['Teammate'].isin(ponte_pokemon_COMPETITIVE['Pokemon'])]

In [523]:
len(teammates_S12VGC['Teammate'].unique().isin)

256

In [542]:
# Elimino trattini tra i nomi
teammates_S12VGC['Pokemon'] = rl.preprocessing.clean(
    teammates_S12VGC['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
teammates_S12VGC['Pokemon'] = teammates_S12VGC['Pokemon'].str.title()

# Elimino trattini tra i nomi
teammates_S12VGC['Teammate'] = rl.preprocessing.clean(
    teammates_S12VGC['Teammate'])
# Rendo Maiuscola la prima lettera di ogni parola
teammates_S12VGC['Teammate'] = teammates_S12VGC['Teammate'].str.title()

Sistemo i nomi sbagliati

In [543]:
teammates_S12VGC['Teammate'].loc[teammates_S12VGC['Teammate'] == 'Tapu20Lele'] = 'Tapu Lele'
teammates_S12VGC['Teammate'].loc[teammates_S12VGC['Teammate'] == 'Tapu20Fini'] = 'Tapu Fini'
teammates_S12VGC['Teammate'].loc[teammates_S12VGC['Teammate'] == 'Mr20Rime'] = 'Mr Rime'
teammates_S12VGC['Teammate'].loc[teammates_S12VGC['Teammate'] == 'Tapu20Koko'] = 'Tapu Koko'

Vado ora a joinare il datataset ponte con questa tabella ponte, generando le tabelle ponte S12VGC ma con i nomi giusti delle varie entità

In [557]:
ponte_pokemon_pokemon_USED_IN_TEAM_WITH = pd.merge(
    teammates_S12VGC,
    ponte_pokemon_COMPETITIVE,
    on='Pokemon',
    how='inner').drop(axis=1, labels='Pokemon').rename(columns={'Name':'Pokemon'})
ponte_pokemon_pokemon_USED_IN_TEAM_WITH.reset_index(drop=True, inplace=True)
ponte_pokemon_pokemon_USED_IN_TEAM_WITH

,Teammate,Use_Percentage (%),Pokemon
0,Incineroar,54.189%,Zacian Crowned Sword
1,Kyogre,31.467%,Zacian Crowned Sword
2,Grimmsnarl,31.148%,Zacian Crowned Sword
3,Rillaboom,29.083%,Zacian Crowned Sword
4,Gastrodon,24.347%,Zacian Crowned Sword
...,...,...,...
3417,Weezing Galar,0.014%,Gourgeist Large Size
3418,Duraludon,0.014%,Gourgeist Large Size
3419,Togetic,0.014%,Gourgeist Large Size
3420,Talonflame,0.014%,Gourgeist Large Size


In [558]:
ponte_pokemon_pokemon_USED_IN_TEAM_WITH = pd.merge(
    ponte_pokemon_pokemon_USED_IN_TEAM_WITH,
    ponte_pokemon_COMPETITIVE,
    left_on='Teammate',
    right_on='Pokemon',
    how='inner').drop(axis=1,
                      labels=['Pokemon_y', 'Teammate']).rename(columns={
                          'Name': 'Teammate',
                          'Pokemon_x': 'Pokemon'
                      })
ponte_pokemon_pokemon_USED_IN_TEAM_WITH.reset_index(drop=True, inplace=True)
ponte_pokemon_pokemon_USED_IN_TEAM_WITH

,Use_Percentage (%),Pokemon,Teammate
0,54.189%,Zacian Crowned Sword,Incineroar
1,37.766%,Kyogre,Incineroar
2,56.035%,Grimmsnarl,Incineroar
3,47.343%,Regieleki,Incineroar
4,61.378%,Rillaboom,Incineroar
...,...,...,...
3406,50.083%,Noctowl,Claydol
3407,6.603%,Corsola,Flygon
3408,0.335%,Mime Jr.,Scrafty
3409,0.014%,Gourgeist Large Size,Scrafty


Salvataggio

In [559]:
ponte_pokemon_pokemon_USED_IN_TEAM_WITH.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_pokemon_USED_IN_TEAM_WITH.csv',
    index=False)

#### `USED_WITH_ITEM`

In [562]:
items_i_df.head(2)

,Name,Effect (alt.),Effect,Usage Attributes,Specific Category,General Category,image_url
0,Ability Capsule,Switches a Pokémon between its two possible (n...,A capsule that allows a Pokémon with two Abili...,<NA>,Vitamins,Hold items,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,<NA>,Changes a Pokémon's ability to its Hidden Abil...,<NA>,Vitamins,General items,https://img.pokemondb.net/s.png


In [560]:
items_S12VGC

,Pokemon,Item,Use_Percentage (%)
0,Zacian-Crowned,Rusted Sword,100.000%
1,Incineroar,Shuca Berry,36.082%
2,Incineroar,Safety Goggles,23.390%
3,Incineroar,Sitrus Berry,14.031%
4,Incineroar,Focus Sash,8.994%
...,...,...,...
1887,Corsola,Other,0.734%
1888,Mime Jr.,Focus Sash,99.860%
1889,Mime Jr.,Other,0.140%
1890,Gourgeist-Large,Figy Berry,99.986%


Pulizia stringhe

In [541]:
#teammates_S12VGC[~teammates_S12VGC['Teammate'].isin(ponte_pokemon_COMPETITIVE['Pokemon'])]

In [563]:
# Elimino trattini tra i nomi
items_S12VGC['Pokemon'] = rl.preprocessing.clean(
    items_S12VGC['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
items_S12VGC['Pokemon'] = items_S12VGC['Pokemon'].str.title()


Vado ora a joinare il datataset ponte con questa tabella ponte, generando le tabelle ponte S12VGC ma con i nomi giusti delle varie entità

In [564]:
ponte_pokemon_item_USED_WITH_ITEM = pd.merge(
    items_S12VGC,
    ponte_pokemon_COMPETITIVE,
    on='Pokemon',
    how='inner').drop(axis=1, labels='Pokemon').rename(columns={'Name':'Pokemon'})
ponte_pokemon_item_USED_WITH_ITEM.reset_index(drop=True, inplace=True)
ponte_pokemon_item_USED_WITH_ITEM

,Item,Use_Percentage (%),Pokemon
0,Rusted Sword,100.000%,Zacian Crowned Sword
1,Shuca Berry,36.082%,Incineroar
2,Safety Goggles,23.390%,Incineroar
3,Sitrus Berry,14.031%,Incineroar
4,Focus Sash,8.994%,Incineroar
...,...,...,...
1887,Other,0.734%,Corsola
1888,Focus Sash,99.860%,Mime Jr.
1889,Other,0.140%,Mime Jr.
1890,Figy Berry,99.986%,Gourgeist Large Size


Salvataggio

In [585]:
ponte_pokemon_item_USED_WITH_ITEM.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_item_USED_WITH_ITEM.csv',
    index=False)

#### `USED_WITH_MOVE`

In [574]:
ponte_move_COMPETITIVE.head()

,Move,Name
0,Behemoth Blade,Behemoth Blade
1,Protect,Protect
2,Sacred Sword,Sacred Sword
3,Play Rough,Play Rough
4,Substitute,Substitute


In [566]:
moves_i_df.head()

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%)
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10
3,Acid Armor,1,Poison,<NA>,<NA>,20,Status,Sharply raises user's Defense.,<NA>
4,Acid Spray,5,Poison,40,100,20,Special,Sharply lowers opponent's Special Defense.,100


In [567]:
moves_S12VGC

,Pokemon,Move,Type,Use_Percentage (%)
0,Zacian-Crowned,Behemoth Blade,steel,99.996%
1,Zacian-Crowned,Protect,normal,98.672%
2,Zacian-Crowned,Sacred Sword,fighting,84.154%
3,Zacian-Crowned,Play Rough,fairy,64.487%
4,Zacian-Crowned,Substitute,normal,30.274%
...,...,...,...,...
3613,Mime Jr.,Other,NaN,12.562%
3614,Gourgeist-Large,Power Whip,grass,100.000%
3615,Gourgeist-Large,Trick Room,psychic,99.986%
3616,Gourgeist-Large,Imprison,psychic,99.986%


Pulizia stringhe

Quelle mancanti sono tutte NaN o Other

In [581]:
moves_S12VGC[~moves_S12VGC['Move'].isin(ponte_move_COMPETITIVE['Move'])]['Move'].unique()

array(['Other', nan], dtype=object)

In [523]:
#len(teammates_S12VGC['Teammate'].unique().isin)

256

In [568]:
# Elimino trattini tra i nomi
moves_S12VGC['Pokemon'] = rl.preprocessing.clean(
    moves_S12VGC['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
moves_S12VGC['Pokemon'] = moves_S12VGC['Pokemon'].str.title()

Sistemo i nomi sbagliati

Vado ora a joinare il datataset ponte con questa tabella ponte, generando le tabelle ponte S12VGC ma con i nomi giusti delle varie entità

In [587]:
ponte_pokemon_move_USED_WITH_MOVE = pd.merge(
    moves_S12VGC,
    ponte_pokemon_COMPETITIVE,
    on='Pokemon',
    how='inner').drop(axis=1, labels='Pokemon').rename(columns={'Name':'Pokemon'})
ponte_pokemon_move_USED_WITH_MOVE.reset_index(drop=True, inplace=True)
ponte_pokemon_move_USED_WITH_MOVE.drop(axis=1, labels='Type', inplace=True)
ponte_pokemon_move_USED_WITH_MOVE

,Move,Use_Percentage (%),Pokemon
0,Behemoth Blade,99.996%,Zacian Crowned Sword
1,Protect,98.672%,Zacian Crowned Sword
2,Sacred Sword,84.154%,Zacian Crowned Sword
3,Play Rough,64.487%,Zacian Crowned Sword
4,Substitute,30.274%,Zacian Crowned Sword
...,...,...,...
3613,Other,12.562%,Mime Jr.
3614,Power Whip,100.000%,Gourgeist Large Size
3615,Trick Room,99.986%,Gourgeist Large Size
3616,Imprison,99.986%,Gourgeist Large Size


In [588]:
ponte_pokemon_move_USED_WITH_MOVE = pd.merge(
    ponte_pokemon_move_USED_WITH_MOVE,
    ponte_move_COMPETITIVE,
    on='Move',
    how='inner')
ponte_pokemon_move_USED_WITH_MOVE.reset_index(drop=True, inplace=True)
ponte_pokemon_move_USED_WITH_MOVE.drop(axis=1,
                      labels=['Move']).rename(columns={
                          'Name': 'Move'
                      }, inplace=True)
ponte_pokemon_move_USED_WITH_MOVE

,Move,Use_Percentage (%),Pokemon,Name
0,Behemoth Blade,99.996%,Zacian Crowned Sword,Behemoth Blade
1,Protect,98.672%,Zacian Crowned Sword,Protect
2,Protect,67.612%,Kyogre,Protect
3,Protect,72.095%,Regieleki,Protect
4,Protect,47.379%,Rillaboom,Protect
...,...,...,...,...
3266,Wrap,23.041%,Lickilicky,Wrap
3267,Thrash,17.229%,Lickilicky,Thrash
3268,Headbutt,33.562%,Dunsparce,Headbutt
3269,Sleep Talk,9.898%,Dunsparce,Sleep Talk


Salvataggio

In [589]:
ponte_pokemon_move_USED_WITH_MOVE.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_move_USED_WITH_MOVE.csv',
    index=False)

#### `COMMON_SPREAD`

In [590]:
natures_i_df.head(2)

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry


In [591]:
EV_spread_S12VGC

,Pokemon,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%)
0,Zacian-Crowned,Jolly,0/252/0/0/4/252,10.388%
1,Zacian-Crowned,Adamant,252/28/4/0/12/212,4.504%
2,Zacian-Crowned,Jolly,4/252/0/0/0/252,4.419%
3,Zacian-Crowned,Adamant,188/164/4/0/4/148,2.891%
4,Zacian-Crowned,Jolly,0/252/4/0/0/252,2.456%
...,...,...,...,...
6814,Gourgeist-Large,Sassy,252/4/0/0/252/0,0.000%
6815,Gourgeist-Large,Relaxed,252/0/252/4/0/0,0.000%
6816,Gourgeist-Large,Quiet,252/0/0/252/4/0,0.000%
6817,Gourgeist-Large,Adamant,252/60/0/0/196/0,0.000%


Pulizia stringhe

In [541]:
#teammates_S12VGC[~teammates_S12VGC['Teammate'].isin(ponte_pokemon_COMPETITIVE['Pokemon'])]

In [592]:
# Elimino trattini tra i nomi
EV_spread_S12VGC['Pokemon'] = rl.preprocessing.clean(
    EV_spread_S12VGC['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
EV_spread_S12VGC['Pokemon'] = EV_spread_S12VGC['Pokemon'].str.title()


Vado ora a joinare il datataset ponte con questa tabella ponte, generando le tabelle ponte S12VGC ma con i nomi giusti delle varie entità

In [593]:
ponte_pokemon_nature_COMMON_SPREAD = pd.merge(
    EV_spread_S12VGC,
    ponte_pokemon_COMPETITIVE,
    on='Pokemon',
    how='inner').drop(axis=1, labels='Pokemon').rename(columns={'Name':'Pokemon'})
ponte_pokemon_nature_COMMON_SPREAD.reset_index(drop=True, inplace=True)
ponte_pokemon_nature_COMMON_SPREAD

,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%),Pokemon
0,Jolly,0/252/0/0/4/252,10.388%,Zacian Crowned Sword
1,Adamant,252/28/4/0/12/212,4.504%,Zacian Crowned Sword
2,Jolly,4/252/0/0/0/252,4.419%,Zacian Crowned Sword
3,Adamant,188/164/4/0/4/148,2.891%,Zacian Crowned Sword
4,Jolly,0/252/4/0/0/252,2.456%,Zacian Crowned Sword
...,...,...,...,...
6814,Sassy,252/4/0/0/252/0,0.000%,Gourgeist Large Size
6815,Relaxed,252/0/252/4/0/0,0.000%,Gourgeist Large Size
6816,Quiet,252/0/0/252/4/0,0.000%,Gourgeist Large Size
6817,Adamant,252/60/0/0/196/0,0.000%,Gourgeist Large Size


Salvataggio

In [594]:
ponte_pokemon_nature_COMMON_SPREAD.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_nature_COMMON_SPREAD.csv',
    index=False)

#### `USED_WITH_ABILITY`

In [595]:
abilities_i_df.head(2)

,Name,Generation,Description
0,Adaptability,4,Powers up moves of the same type.
1,Aerilate,6,Turns Normal-type moves into Flying-type moves.


In [596]:
abilities_S12VGC

,Pokemon,Ability,Use_Percentage (%)
0,Zacian-Crowned,Intrepid Sword,100.000%
1,Incineroar,Intimidate,99.927%
2,Incineroar,Blaze,0.073%
3,Kyogre,Drizzle,100.000%
4,Grimmsnarl,Prankster,99.959%
...,...,...,...
768,Mime Jr.,Soundproof,99.870%
769,Mime Jr.,Filter,0.130%
770,Mime Jr.,Technician,0.000%
771,Gourgeist-Large,Frisk,100.000%


Pulizia stringhe

In [541]:
#teammates_S12VGC[~teammates_S12VGC['Teammate'].isin(ponte_pokemon_COMPETITIVE['Pokemon'])]

In [597]:
# Elimino trattini tra i nomi
abilities_S12VGC['Pokemon'] = rl.preprocessing.clean(
    abilities_S12VGC['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
abilities_S12VGC['Pokemon'] = abilities_S12VGC['Pokemon'].str.title()


Vado ora a joinare il datataset ponte con questa tabella ponte, generando le tabelle ponte S12VGC ma con i nomi giusti delle varie entità

In [598]:
ponte_pokemon_nature_USED_WITH_ABILITY = pd.merge(
    abilities_S12VGC,
    ponte_pokemon_COMPETITIVE,
    on='Pokemon',
    how='inner').drop(axis=1, labels='Pokemon').rename(columns={'Name':'Pokemon'})
ponte_pokemon_nature_USED_WITH_ABILITY.reset_index(drop=True, inplace=True)
ponte_pokemon_nature_USED_WITH_ABILITY

,Ability,Use_Percentage (%),Pokemon
0,Intrepid Sword,100.000%,Zacian Crowned Sword
1,Intimidate,99.927%,Incineroar
2,Blaze,0.073%,Incineroar
3,Drizzle,100.000%,Kyogre
4,Prankster,99.959%,Grimmsnarl
...,...,...,...
767,Soundproof,99.870%,Mime Jr.
768,Filter,0.130%,Mime Jr.
769,Technician,0.000%,Mime Jr.
770,Frisk,100.000%,Gourgeist Large Size


Salvataggio

In [599]:
ponte_pokemon_nature_USED_WITH_ABILITY.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_nature_USED_WITH_ABILITY.csv',
    index=False)